Dokumentation Game Entwicklung

Um eine solide Basis für das Spiel zu bekommen, haben wir als Erstes die Datei mit den Vorgaben hochgeladen und den Prompt "Erstelle anhand dieser Vorgaben ein Spiel. Das Spiel soll wie Mario aufgebaut sein, es soll 4 Level haben. In den Leveln sollen sich Gegner befinden, die Schwierigkeit soll sich außerdem steigern."

In [ ]:
import java.awt.*;

public class Enemy {
    private final int w = 32, h = 32;
    private final Level level;
    private double x, y;
    private double vx = 1.2;

    public Enemy(double x, double y, Level level) {
        this.x = x;
        this.y = y;
        this.level = level;
    }

    public void update() {

        // ==========================
        // 1) GRAVITY (immer zuerst!)
        // ==========================
        double nextY = y + 4;  // Fallgeschwindigkeit

        boolean onGround = false;
        for (Tile t : level.getSolidTiles()) {
            Rectangle test = new Rectangle((int) x, (int) nextY, w, h);
            if (test.intersects(t.getRect())) {
                // auf Boden setzen
                y = t.getY() - h;
                onGround = true;
                break;
            }
        }

        if (!onGround) {
            // frei fallender Gegner → FALLEN
            y = nextY;
            return; // solange er fällt NICHTS anderes tun
        }


        // =========================================
        // 2) ANTI-FALL-KANTEN-ERKENNUNG (nur wenn Boden!)
        // =========================================
        int frontX = (int) (x + (vx > 0 ? w + 2 : -2));
        int belowY = (int) (y + h + 2);

        boolean groundAhead = false;
        for (Tile t : level.getSolidTiles()) {
            if (t.getRect().contains(frontX, belowY)) {
                groundAhead = true;
                break;
            }
        }

        if (!groundAhead) {
            // keine Kante → umdrehen
            vx = -vx;
            return;
        }


        // ==========================
        // 3) HORIZONTALE BEWEGUNG
        // ==========================
        double nextX = x + vx;
        Rectangle future = new Rectangle((int) nextX, (int) y, w, h);

        Tile blockingTile = null;
        for (Tile t : level.getSolidTiles()) {
            if (future.intersects(t.getRect())) {
                blockingTile = t;
                break;
            }
        }

        if (blockingTile != null) {
            // HOCHKLETTERN VERHINDERN
            if (blockingTile.getY() < y + h - 4) {
                vx = -vx;
                return;
            }
            // normale Kollision
            vx = -vx;
            return;
        }

        // Wenn alles ok → bewegen
        x = nextX;
    }


    public Rectangle getBounds() {
        return new Rectangle((int) Math.round(x), (int) Math.round(y), w, h);
    }

    public int getY() {
        return (int) Math.round(y);
    }

    public void draw(Graphics2D g, int camX) {
        int dx = (int) Math.round(x) - camX;
        int dy = (int) Math.round(y);

        g.setColor(new Color(255, 140, 0));
        g.fillOval(dx, dy, w, h);
        g.setColor(Color.BLACK);
        g.drawOval(dx, dy, w, h);
    }
}

In [ ]:
import javax.swing.JFrame;

public class Game {

    public static void createAndShowGUI() {
        JFrame frame = new JFrame("Platformer — Mario-Style (Simple)");
        frame.setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);
        frame.setResizable(false);

        GamePanel panel = new GamePanel();
        frame.setContentPane(panel);
        frame.pack();
        frame.setLocationRelativeTo(null);
        frame.setVisible(true);

        panel.startGame();
    }
}

In [ ]:
import javax.swing.*;
import java.awt.*;
import java.awt.event.ActionEvent;
import java.util.ArrayList;

public class GamePanel extends JPanel implements java.awt.event.ActionListener {
    public static final int WIDTH = 800;
    public static final int HEIGHT = 600;

    private final Timer timer;
    private boolean left, right;

    private Player player;
    private ArrayList<Enemy> enemies;
    private Level level;
    private int currentLevelIndex = 0;
    private int score = 0;
    private static int lives = 3;
    private static State state = State.START;

    public GamePanel() {
        setPreferredSize(new Dimension(WIDTH, HEIGHT));
        setBackground(Color.CYAN);
        setFocusable(true);

        timer = new Timer(16, this); // ~60 FPS
        setupKeyBindings();

        loadLevel(currentLevelIndex);
    }

    public void startGame() {
        timer.start();
    }

    private void loadLevel(int index) {
        level = Level.createSampleLevel(index);
        player = new Player(50, 450 - 48, level); // start slightly above ground
        enemies = new ArrayList<>();
        for (int[] p : level.getEnemyPositions()) {
            enemies.add(new Enemy(p[0], p[1], level));
        }
        state = State.START;
    }

    private void setupKeyBindings() {
        InputMap im = getInputMap(JComponent.WHEN_IN_FOCUSED_WINDOW);
        ActionMap am = getActionMap();

        // LEFT / A
        im.put(KeyStroke.getKeyStroke("pressed LEFT"), "left_pressed");
        im.put(KeyStroke.getKeyStroke("released LEFT"), "left_released");
        im.put(KeyStroke.getKeyStroke("pressed A"), "left_pressed");
        im.put(KeyStroke.getKeyStroke("released A"), "left_released");
        am.put("left_pressed", new AbstractAction() { public void actionPerformed(ActionEvent e) { left = true; }});
        am.put("left_released", new AbstractAction() { public void actionPerformed(ActionEvent e) { left = false; }});

        // RIGHT / D
        im.put(KeyStroke.getKeyStroke("pressed RIGHT"), "right_pressed");
        im.put(KeyStroke.getKeyStroke("released RIGHT"), "right_released");
        im.put(KeyStroke.getKeyStroke("pressed D"), "right_pressed");
        im.put(KeyStroke.getKeyStroke("released D"), "right_released");
        am.put("right_pressed", new AbstractAction() { public void actionPerformed(ActionEvent e) { right = true; }});
        am.put("right_released", new AbstractAction() { public void actionPerformed(ActionEvent e) { right = false; }});

        // JUMP: SPACE / UP / W
        im.put(KeyStroke.getKeyStroke("pressed SPACE"), "jump_pressed");
        im.put(KeyStroke.getKeyStroke("pressed UP"), "jump_pressed");
        im.put(KeyStroke.getKeyStroke("pressed W"), "jump_pressed");
        im.put(KeyStroke.getKeyStroke("released SPACE"), "jump_released");
        im.put(KeyStroke.getKeyStroke("released UP"), "jump_released");
        im.put(KeyStroke.getKeyStroke("released W"), "jump_released");
        am.put("jump_pressed", new AbstractAction() { public void actionPerformed(ActionEvent e) { if (player != null) player.pressJump(); }});
        am.put("jump_released", new AbstractAction() { public void actionPerformed(ActionEvent e) { if (player != null) player.releaseJump(); }});

        // ENTER: start / next
        im.put(KeyStroke.getKeyStroke("pressed ENTER"), "enter_pressed");
        am.put("enter_pressed", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                if (state == State.START) state = State.RUNNING;
                else if (state == State.LEVEL_COMPLETE) {
                    currentLevelIndex++;
                    if (currentLevelIndex >= Level.NUM_LEVELS) {
                        state = State.GAME_OVER;
                    } else {
                        loadLevel(currentLevelIndex);
                        state = State.START;
                    }
                }
            }
        });

        // R: restart after game over
        im.put(KeyStroke.getKeyStroke("pressed R"), "r_pressed");
        am.put("r_pressed", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                if (state == State.GAME_OVER) {
                    score = 0; lives = 3; currentLevelIndex = 0; loadLevel(currentLevelIndex); state = State.START;
                }
            }
        });
    }

    @Override
    public void actionPerformed(ActionEvent e) {
        double dt = timer.getDelay() / 1000.0;

        if (state == State.RUNNING) {
            if (left && !right) player.moveLeft();
            else if (right && !left) player.moveRight();
            else player.stopHorizontal();

            player.update(dt);
            for (Enemy en : new ArrayList<>(enemies)) en.update();

            // enemy-player collisions
            for (Enemy en : new ArrayList<>(enemies)) {
                if (player.getBounds().intersects(en.getBounds())) {
                    if (player.isFalling() && player.getY() < en.getY()) {
                        enemies.remove(en);
                        score += 100;
                        player.bounceAfterStomp();
                    } else {
                        lives--;
                        if (lives <= 0) state = State.GAME_OVER;
                        else player.respawn();
                    }
                }
            }

            if (level.isEndReached(player)) {
                state = State.LEVEL_COMPLETE;
            }
        }

        repaint();
    }

    @Override
    protected void paintComponent(Graphics g) {
        super.paintComponent(g);
        Graphics2D g2 = (Graphics2D) g;

        int camX = 0;
        if (player != null) {
            int px = player.getBounds().x;
            camX = px - WIDTH / 2;
            if (camX < 0) camX = 0;
            if (camX > level.getWidth() - WIDTH) camX = level.getWidth() - WIDTH;
        }

        level.draw(g2, camX);

        if (player != null) player.draw(g2, camX);
        for (Enemy en : new ArrayList<>(enemies)) en.draw(g2, camX);

        // HUD
        g2.setColor(Color.BLACK);
        g2.setFont(new Font("SansSerif", Font.BOLD, 18));
        g2.drawString("Score: " + score, 10, 20);
        g2.drawString("Lives: " + lives, 10, 40);
        g2.drawString("Level: " + (currentLevelIndex + 1), 700, 20);

        if (state == State.START) {
            drawCenteredString(g2, "PRESS ENTER TO START", getWidth(), getHeight());
            drawCenteredString(g2, "Controls: Arrow keys or WASD to move, SPACE/W to jump", getWidth(), getHeight() - 50);
        } else if (state == State.GAME_OVER) {
            drawCenteredString(g2, "GAME OVER - Press R to restart", getWidth(), getHeight());
        } else if (state == State.LEVEL_COMPLETE) {
            drawCenteredString(g2, "LEVEL COMPLETE - Press ENTER to continue", getWidth(), getHeight());
        }
    }

    public static void  subLive(){
        lives--;
        if (lives <= 0) state = State.GAME_OVER;
    }

    private void drawCenteredString(Graphics2D g, String text, int w, int h) {
        FontMetrics fm = g.getFontMetrics();
        int x = (w - fm.stringWidth(text)) / 2;
        int y = (h - fm.getHeight()) / 2 + fm.getAscent();
        g.setColor(Color.WHITE);
        g.fillRect(x - 10, y - fm.getAscent() - 5, fm.stringWidth(text) + 20, fm.getHeight() + 10);
        g.setColor(Color.BLACK);
        g.drawString(text, x, y);
    }

    private enum State { START, RUNNING, GAME_OVER, LEVEL_COMPLETE }
}

In [ ]:
import java.awt.*;
import java.util.ArrayList;

public class Level {
    public static final int TILE_SIZE = 50;
    public static final int NUM_LEVELS = 4;

    private int width;
    private int height;
    private ArrayList<Tile> solidTiles = new ArrayList<>();
    private int[][] enemyPositions;
    private int endX, endY;

    public Level(int width, int height) {
        this.width = width;
        this.height = height;
    }

    public static Level createSampleLevel(int index) {
        Level level = new Level(2500, 600);
        ArrayList<Tile> tiles = new ArrayList<>();
        int groundY = 550;

        // Grundboden mit kleinen Lücken
        for (int x = 0; x < 50; x++) {
            if (x % 8 == 4 && index > 0) continue; // kleine Lücken in höheren Levels
            tiles.add(new Tile(x * TILE_SIZE, groundY, TILE_SIZE, TILE_SIZE));
        }

        // -----------------------
        // LEVEL 1 – einfaches Springen & Bewegung
        // -----------------------
        if (index == 0) {
            tiles.add(new Tile(400, 500, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(500, 450, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(600, 400, TILE_SIZE, TILE_SIZE));

            // neue Sektion 2 (nach 900px)
            tiles.add(new Tile(950, 500, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(1050, 450, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(1150, 400, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(1250, 350, TILE_SIZE, TILE_SIZE));

            // Endzone mit leichter Rampe
            for (int x = 38; x < 45; x++) {
                tiles.add(new Tile(x * TILE_SIZE, groundY - (x - 38) * 10, TILE_SIZE, TILE_SIZE));
            }

            level.enemyPositions = new int[][] {
                    {700, 500}, {1100, 500}, {1500, 500}
            };

            level.endX = 2300;
            level.endY = groundY - TILE_SIZE;
        }

        // -----------------------
        // LEVEL 2 – Doppelsprünge, Hindernisse, kleine Gruben
        // -----------------------
        else if (index == 1) {
            tiles.add(new Tile(400, 500, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(500, 450, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(600, 400, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(750, 400, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(900, 350, TILE_SIZE, TILE_SIZE));

            // zweite Hälfte mit erhöhten Plattformen
            for (int i = 0; i < 5; i++) {
                tiles.add(new Tile(1200 + i * 70, 450 - (i % 2) * 50, TILE_SIZE, TILE_SIZE));
            }

            tiles.add(new Tile(1650, 350, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(1750, 400, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(1900, 450, TILE_SIZE, TILE_SIZE));

            level.enemyPositions = new int[][] {
                    {500, 500}, {850, 500}, {1300, 500}, {1700, 500}
            };

            level.endX = 2400;
            level.endY = groundY - TILE_SIZE;
        }

        // -----------------------
        // LEVEL 3 – Gegnerpfade & anspruchsvollere Sprünge
        // -----------------------
        else if (index == 2) {
            // erste Sektion
            tiles.add(new Tile(400, 500, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(500, 450, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(600, 400, TILE_SIZE, TILE_SIZE));

            // tiefe Lücke + Plattform darüber
            for (int x = 13; x < 17; x++) {
                int finalX = x;
                tiles.removeIf(t -> t.getX() / TILE_SIZE == finalX);
            }
            tiles.add(new Tile(700, 350, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(800, 300, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(900, 350, TILE_SIZE, TILE_SIZE));

            // zweite Sektion – gestaffelte Höhen
            tiles.add(new Tile(1200, 450, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(1300, 400, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(1400, 350, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(1500, 400, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(1600, 450, TILE_SIZE, TILE_SIZE));

            level.enemyPositions = new int[][] {
                    {600, 500}, {1000, 500}, {1450, 500}, {1800, 500}
            };

            level.endX = 2400;
            level.endY = groundY - TILE_SIZE;
        }

        // -----------------------
        // LEVEL 4 – Finale: mehr Vertikalität & lange Sprungpassagen
        // -----------------------
        else if (index == 3) {
            tiles.add(new Tile(400, 500, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(500, 450, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(600, 400, TILE_SIZE, TILE_SIZE));
            tiles.add(new Tile(750, 350, TILE_SIZE, TILE_SIZE));

            // mittlere Plattform-Linie
            for (int i = 0; i < 7; i++) {
                tiles.add(new Tile(1000 + i * 100, 400 - (i % 2) * 50, TILE_SIZE, TILE_SIZE));
            }

            // finale Rampe hoch zur Flagge
            for (int i = 0; i < 6; i++) {
                tiles.add(new Tile(1800 + i * 50, groundY - i * 30, TILE_SIZE, TILE_SIZE));
            }

            level.enemyPositions = new int[][] {
                    {650, 500}, {1100, 500}, {1400, 500}, {1700, 500}, {2000, 500}
            };

            level.endX = 2450;
            level.endY = groundY - 150;
        }

        level.solidTiles = tiles;
        return level;
    }

    public boolean isEndReached(Player p) {
        Rectangle flagRect = new Rectangle(endX, endY - 150, 40, 200);
        return p.getBounds().intersects(flagRect);
    }

    public void draw(Graphics2D g, int camX) {
        // Himmel
        g.setColor(new Color(135, 206, 250));
        g.fillRect(0, 0, width, height);

        // Boden
        g.setColor(new Color(80, 80, 80));
        for (Tile t : solidTiles) {
            g.fillRect(t.getX() - camX, t.getY(), t.getW(), t.getH());
        }

        // Flagge
        g.setColor(new Color(50, 50, 50));
        g.fillRect(endX - camX - 10, endY + TILE_SIZE - 20, 60, 20);

        g.setColor(new Color(200, 200, 200));
        g.fillRect(endX - camX, endY + TILE_SIZE - 20 - 150, 5, 150);

        g.setColor(Color.RED);
        g.fillRect(endX - camX + 5, endY + TILE_SIZE - 20 - 150, 40, 25);
        g.setColor(Color.BLACK);
        g.drawRect(endX - camX + 5, endY + TILE_SIZE - 20 - 150, 40, 25);
    }

    public ArrayList<Tile> getSolidTiles() { return solidTiles; }
    public int[][] getEnemyPositions() { return enemyPositions; }
    public int getWidth() { return width; }
    public int getHeight() { return height; }
}

In [ ]:
import javax.swing.SwingUtilities;

public class Main {
    public static void main(String[] args) {
        SwingUtilities.invokeLater(() -> Game.createAndShowGUI());
    }
}

In [ ]:
import java.awt.*;

public class Tile {
    private int x, y, w, h;

    public Tile(int x, int y, int w, int h) {
        this.x = x;
        this.y = y;
        this.w = w;
        this.h = h;
    }

    public void draw(Graphics2D g, int camX) {
        g.setColor(new Color(150, 75, 0)); // brauner Boden
        g.fillRect(x - camX, y, w, h);
        g.setColor(Color.BLACK);
        g.drawRect(x - camX, y, w, h);
    }

    public Rectangle getRect() {
        return new Rectangle(x, y, w, h);
    }

    public int getX() { return x; }
    public int getY() { return y; }
    public int getW() { return w; }
    public int getH() { return h; }
}

Um das Game interessanter zu gestalten wollten wir den Spieler, die Gegner, die Blöcke, den Boden und den Hintergrund mit eigenen Texturen anzeigen lassen. Außerdem haben wir mit ESC eine einfache Option das Spiel zu schließen implementiert. 

Als Prompts haben wir einmal "Implementiere ESC als Taste, um das Spiel zu schließen und das Programm somit zu beenden." verwendet und für die Bilder: "Füge für den Spieler, die Gegner, die Blöcke, den Boden und den Hintergrund die Option hinzu Bilder einzufügen. Wenn keine Bilder vorhanden sein sollten, soll das ursprüngliche Design verwendet werden.

Nach der Kombination dieser Änderung haben wir folgenden Code erhalten:

In [ ]:
import javax.imageio.ImageIO;
import java.awt.*;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;

public class Enemy {
    private final int w = 32, h = 32;
    private final Level level;
    private double x, y;
    private double vx = 1.2;
    private BufferedImage sprite;

    public Enemy(double x, double y, Level level) {
        this.x = x;
        this.y = y;
        this.level = level;
        try{
            sprite = ImageIO.read(new File("res/enemy.png"));
        } catch (IOException e) {
            System.err.println("Enemy sprite loading failed.");
        }
    }

    public void update() {

        // ==========================
        // 1) GRAVITY (immer zuerst!)
        // ==========================
        double nextY = y + 4;  // Fallgeschwindigkeit

        boolean onGround = false;
        for (Tile t : level.getSolidTiles()) {
            Rectangle test = new Rectangle((int) x, (int) nextY, w, h);
            if (test.intersects(t.getRect())) {
                // auf Boden setzen
                y = t.getY() - h;
                onGround = true;
                break;
            }
        }

        if (!onGround) {
            // frei fallender Gegner → FALLEN
            y = nextY;
            return; // solange er fällt NICHTS anderes tun
        }


        // =========================================
        // 2) ANTI-FALL-KANTEN-ERKENNUNG (nur wenn Boden!)
        // =========================================
        int frontX = (int) (x + (vx > 0 ? w + 2 : -2));
        int belowY = (int) (y + h + 2);

        boolean groundAhead = false;
        for (Tile t : level.getSolidTiles()) {
            if (t.getRect().contains(frontX, belowY)) {
                groundAhead = true;
                break;
            }
        }

        if (!groundAhead) {
            // keine Kante → umdrehen
            vx = -vx;
            return;
        }


        // ==========================
        // 3) HORIZONTALE BEWEGUNG
        // ==========================
        double nextX = x + vx;
        Rectangle future = new Rectangle((int) nextX, (int) y, w, h);

        Tile blockingTile = null;
        for (Tile t : level.getSolidTiles()) {
            if (future.intersects(t.getRect())) {
                blockingTile = t;
                break;
            }
        }

        if (blockingTile != null) {
            // HOCHKLETTERN VERHINDERN
            if (blockingTile.getY() < y + h - 4) {
                vx = -vx;
                return;
            }
            // normale Kollision
            vx = -vx;
            return;
        }

        // Wenn alles ok → bewegen
        x = nextX;
    }


    public Rectangle getBounds() {
        return new Rectangle((int) Math.round(x), (int) Math.round(y), w, h);
    }

    public int getY() {
        return (int) Math.round(y);
    }

    public void draw(Graphics2D g, int camX) {
        int dx = (int) Math.round(x) - camX;
        int dy = (int) Math.round(y);

        if (sprite != null) {
            g.drawImage(sprite, dx, dy, w, h, null);
        } else {
            g.setColor(new Color(255, 140, 0));
            g.fillOval(dx, dy, w, h);
            g.setColor(Color.BLACK);
            g.drawOval(dx, dy, w, h);
        }
    }
}

In [ ]:
import javax.swing.*;

public class Game {

    public static void createAndShowGUI() {
        JFrame frame = new JFrame("Platformer — Mario-Style (Simple)");
        frame.setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);
        frame.setResizable(false);

        GamePanel panel = new GamePanel();
        frame.setContentPane(panel);
        frame.pack();
        frame.setLocationRelativeTo(null);
        frame.setVisible(true);

        panel.startGame();
    }
}

In [ ]:
import javax.swing.*;
import java.awt.*;
import java.awt.event.ActionEvent;
import java.awt.event.KeyEvent;
import java.util.ArrayList;

public class GamePanel extends JPanel implements java.awt.event.ActionListener {
    public static final int WIDTH = 800;
    public static final int HEIGHT = 600;
    private static int lives = 3;
    private static State state = State.START;
    private final Timer timer;
    private boolean left, right;
    private Player player;
    private ArrayList<Enemy> enemies;
    private Level level;
    private int currentLevelIndex = 0;
    private int score = 0;

    public GamePanel() {
        setPreferredSize(new Dimension(WIDTH, HEIGHT));
        setBackground(Color.CYAN);
        setFocusable(true);

        timer = new Timer(16, this); // ~60 FPS
        setupKeyBindings();

        loadLevel(currentLevelIndex);
    }

    public static void subLive() {
        lives--;
        if (lives <= 0) state = State.GAME_OVER;
    }

    public void startGame() {
        timer.start();
    }

    private void loadLevel(int index) {
        level = Level.createSampleLevel(index);
        player = new Player(50, 450 - 48, level); // start slightly above ground
        player.loadSprite("res/player.png");
        enemies = new ArrayList<>();
        for (int[] p : level.getEnemyPositions()) {
            enemies.add(new Enemy(p[0], p[1], level));
        }
        state = State.START;
    }

    private void setupKeyBindings() {
        InputMap im = getInputMap(JComponent.WHEN_IN_FOCUSED_WINDOW);
        ActionMap am = getActionMap();

        // LEFT / A
        im.put(KeyStroke.getKeyStroke("pressed LEFT"), "left_pressed");
        im.put(KeyStroke.getKeyStroke("released LEFT"), "left_released");
        im.put(KeyStroke.getKeyStroke("pressed A"), "left_pressed");
        im.put(KeyStroke.getKeyStroke("released A"), "left_released");
        am.put("left_pressed", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                left = true;
            }
        });
        am.put("left_released", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                left = false;
            }
        });

        // RIGHT / D
        im.put(KeyStroke.getKeyStroke("pressed RIGHT"), "right_pressed");
        im.put(KeyStroke.getKeyStroke("released RIGHT"), "right_released");
        im.put(KeyStroke.getKeyStroke("pressed D"), "right_pressed");
        im.put(KeyStroke.getKeyStroke("released D"), "right_released");
        am.put("right_pressed", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                right = true;
            }
        });
        am.put("right_released", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                right = false;
            }
        });

        // JUMP: SPACE / UP / W
        im.put(KeyStroke.getKeyStroke("pressed SPACE"), "jump_pressed");
        im.put(KeyStroke.getKeyStroke("pressed UP"), "jump_pressed");
        im.put(KeyStroke.getKeyStroke("pressed W"), "jump_pressed");
        im.put(KeyStroke.getKeyStroke("released SPACE"), "jump_released");
        im.put(KeyStroke.getKeyStroke("released UP"), "jump_released");
        im.put(KeyStroke.getKeyStroke("released W"), "jump_released");
        am.put("jump_pressed", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                if (player != null) player.pressJump();
            }
        });
        am.put("jump_released", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                if (player != null) player.releaseJump();
            }
        });

        // ENTER: start / next
        im.put(KeyStroke.getKeyStroke("pressed ENTER"), "enter_pressed");
        am.put("enter_pressed", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                if (state == State.START) state = State.RUNNING;
                else if (state == State.LEVEL_COMPLETE) {
                    currentLevelIndex++;
                    if (currentLevelIndex >= Level.NUM_LEVELS) {
                        state = State.GAME_OVER;
                    } else {
                        loadLevel(currentLevelIndex);
                        state = State.START;
                    }
                }
            }
        });

        // R: restart after game over
        im.put(KeyStroke.getKeyStroke("pressed R"), "r_pressed");
        am.put("r_pressed", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                if (state == State.GAME_OVER) {
                    score = 0;
                    lives = 3;
                    currentLevelIndex = 0;
                    loadLevel(currentLevelIndex);
                    state = State.START;
                }
            }
        });

        // ESC: exit game
        im.put(KeyStroke.getKeyStroke(KeyEvent.VK_ESCAPE, 0), "ESC_pressed");
        am.put("ESC_pressed", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                System.exit(0);
            }
        });
    }

    @Override
    public void actionPerformed(ActionEvent e) {
        double dt = timer.getDelay() / 1000.0;

        if (state == State.RUNNING) {
            if (left && !right) player.moveLeft();
            else if (right && !left) player.moveRight();
            else player.stopHorizontal();

            player.update(dt);
            for (Enemy en : new ArrayList<>(enemies)) en.update();

            // enemy-player collisions
            for (Enemy en : new ArrayList<>(enemies)) {
                if (player.getBounds().intersects(en.getBounds())) {
                    if (player.isFalling() && player.getY() < en.getY()) {
                        enemies.remove(en);
                        score += 100;
                        player.bounceAfterStomp();
                    } else {
                        lives--;
                        if (lives <= 0) state = State.GAME_OVER;
                        else player.respawn();
                    }
                }
            }

            if (level.isEndReached(player)) {
                state = State.LEVEL_COMPLETE;
            }
        }

        repaint();
    }

    @Override
    protected void paintComponent(Graphics g) {
        super.paintComponent(g);
        Graphics2D g2 = (Graphics2D) g;

        int camX = 0;
        if (player != null) {
            int px = player.getBounds().x;
            camX = px - WIDTH / 2;
            if (camX < 0) camX = 0;
            if (camX > level.getWidth() - WIDTH) camX = level.getWidth() - WIDTH;
        }

        level.draw(g2, camX);

        if (player != null) player.draw(g2, camX);
        for (Enemy en : new ArrayList<>(enemies)) en.draw(g2, camX);

        // HUD
        g2.setColor(Color.BLACK);
        g2.setFont(new Font("SansSerif", Font.BOLD, 18));
        g2.drawString("Score: " + score, 10, 20);
        g2.drawString("Lives: " + lives, 10, 40);
        g2.drawString("Level: " + (currentLevelIndex + 1), 700, 20);

        if (state == State.START) {
            drawCenteredString(g2, "PRESS ENTER TO START", getWidth(), getHeight());
            drawCenteredString(g2, "Controls: Arrow keys or WASD to move, SPACE/W to jump", getWidth(), getHeight() - 50);
        } else if (state == State.GAME_OVER) {
            drawCenteredString(g2, "GAME OVER - Press R to restart", getWidth(), getHeight());
        } else if (state == State.LEVEL_COMPLETE) {
            drawCenteredString(g2, "LEVEL COMPLETE - Press ENTER to continue", getWidth(), getHeight());
        }
    }

    private void drawCenteredString(Graphics2D g, String text, int w, int h) {
        FontMetrics fm = g.getFontMetrics();
        int x = (w - fm.stringWidth(text)) / 2;
        int y = (h - fm.getHeight()) / 2 + fm.getAscent();
        g.setColor(Color.WHITE);
        g.fillRect(x - 10, y - fm.getAscent() - 5, fm.stringWidth(text) + 20, fm.getHeight() + 10);
        g.setColor(Color.BLACK);
        g.drawString(text, x, y);
    }

    private enum State {START, RUNNING, GAME_OVER, LEVEL_COMPLETE}
}

In [ ]:
import javax.imageio.ImageIO;
import java.awt.*;
import java.awt.image.BufferedImage;
import java.io.File;
import java.util.ArrayList;

public class Level {
    public static final int TILE_SIZE = 50;
    public static final int NUM_LEVELS = 4;
    private static BufferedImage groundTexture;
    private static BufferedImage blockTexture;
    private static BufferedImage background;
    private final int width;
    private final int height;
    private ArrayList<Tile> solidTiles = new ArrayList<>();
    private int[][] enemyPositions;
    private int endX, endY;

    public Level(int width, int height) {
        this.width = width;
        this.height = height;

        try {
            groundTexture = ImageIO.read(new File("res/ground.png"));
        } catch (Exception e) {
            System.err.println("Ground texture loading failed.");
        }

        try {
            blockTexture = ImageIO.read(new File("res/block.png"));
        } catch (Exception e) {
            System.err.println("Block texture loading failed.");
        }
        try {
            background = ImageIO.read(new File("res/background.png"));
        } catch (Exception e) {
            System.err.println("Background texture loading failed.");
        }
    }

    public static Level createSampleLevel(int index) {
        Level level = new Level(2500, 600);
        ArrayList<Tile> tiles = new ArrayList<>();
        int groundY = 550;

        // Grundboden mit kleinen Lücken
        for (int x = 0; x < 50; x++) {
            if (x % 8 == 4 && index > 0) continue; // kleine Lücken in höheren Levels
            tiles.add(new Tile(x * TILE_SIZE, groundY, TILE_SIZE, TILE_SIZE, groundTexture));
        }

        // -----------------------
        // LEVEL 1 – einfaches Springen & Bewegung
        // -----------------------
        if (index == 0) {
            tiles.add(new Tile(400, 500, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(500, 450, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(600, 400, TILE_SIZE, TILE_SIZE, blockTexture));

            // neue Sektion 2 (nach 900px)
            tiles.add(new Tile(950, 500, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(1050, 450, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(1150, 400, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(1250, 350, TILE_SIZE, TILE_SIZE, blockTexture));

            // Endzone mit leichter Rampe
            for (int x = 38; x < 45; x++) {
                tiles.add(new Tile(x * TILE_SIZE, groundY - (x - 38) * 10, TILE_SIZE, TILE_SIZE, groundTexture));
            }

            level.enemyPositions = new int[][]{{700, 500}, {1100, 500}, {1500, 500}};

            level.endX = 2300;
            level.endY = groundY - TILE_SIZE;
        }

        // -----------------------
        // LEVEL 2 – Doppelsprünge, Hindernisse, kleine Gruben
        // -----------------------
        else if (index == 1) {
            tiles.add(new Tile(400, 500, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(500, 450, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(600, 400, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(750, 400, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(900, 350, TILE_SIZE, TILE_SIZE, blockTexture));

            // zweite Hälfte mit erhöhten Plattformen
            for (int i = 0; i < 5; i++) {
                tiles.add(new Tile(1200 + i * 70, 450 - (i % 2) * 50, TILE_SIZE, TILE_SIZE, blockTexture));
            }

            tiles.add(new Tile(1650, 350, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(1750, 400, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(1900, 450, TILE_SIZE, TILE_SIZE, blockTexture));

            level.enemyPositions = new int[][]{{500, 500}, {850, 500}, {1300, 500}, {1700, 500}};

            level.endX = 2400;
            level.endY = groundY - TILE_SIZE;
        }

        // -----------------------
        // LEVEL 3 – Gegnerpfade & anspruchsvollere Sprünge
        // -----------------------
        else if (index == 2) {
            // erste Sektion
            tiles.add(new Tile(400, 500, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(500, 450, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(600, 400, TILE_SIZE, TILE_SIZE, blockTexture));

            // tiefe Lücke + Plattform darüber
            for (int x = 13; x < 17; x++) {
                int finalX = x;
                tiles.removeIf(t -> t.getX() / TILE_SIZE == finalX);
            }
            tiles.add(new Tile(700, 350, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(800, 300, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(900, 350, TILE_SIZE, TILE_SIZE, blockTexture));

            // zweite Sektion – gestaffelte Höhen
            tiles.add(new Tile(1200, 450, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(1300, 400, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(1400, 350, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(1500, 400, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(1600, 450, TILE_SIZE, TILE_SIZE, blockTexture));

            level.enemyPositions = new int[][]{{550, 500}, {1100, 500}, {1450, 500}, {1900, 500}};

            level.endX = 2400;
            level.endY = groundY - TILE_SIZE;
        }

        // -----------------------
        // LEVEL 4 – Finale: mehr Vertikalität & lange Sprungpassagen
        // -----------------------
        else if (index == 3) {
            tiles.add(new Tile(400, 500, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(500, 450, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(600, 400, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(750, 350, TILE_SIZE, TILE_SIZE, blockTexture));

            // mittlere Plattform-Linie
            for (int i = 0; i < 7; i++) {
                tiles.add(new Tile(1000 + i * 100, 400 - (i % 2) * 50, TILE_SIZE, TILE_SIZE, blockTexture));
            }

            // finale Rampe hoch zur Flagge
            for (int i = 0; i < 6; i++) {
                tiles.add(new Tile(1800 + i * 50, groundY - i * 30, TILE_SIZE, TILE_SIZE, blockTexture));
            }

            level.enemyPositions = new int[][]{{650, 500}, {1100, 500}, {1400, 500}, {1700, 500}, {2000, 500}};

            level.endX = 2450;
            level.endY = groundY - 150;
        }

        level.solidTiles = tiles;
        return level;
    }

    public boolean isEndReached(Player p) {
        Rectangle flagRect = new Rectangle(endX, endY - 150, 40, 200);
        return p.getBounds().intersects(flagRect);
    }

    public void draw(Graphics2D g, int camX) {
        // Himmel
        if (background != null) {
            // Hintergrund wiederholen oder strecken
            for (int x = 0; x < width; x += background.getWidth()) {
                g.drawImage(background, x - camX / 2, 0, null);
                // camX/2 für Parallax-Effekt
            }
        } else {
            g.setColor(new Color(135, 206, 250));
            g.fillRect(0, 0, width, height);
        }


        // Boden
        for (Tile t : solidTiles) {
            if (groundTexture != null) {
                t.draw(g, camX);
                //g.drawImage(groundTexture, t.getX() - camX, t.getY(), t.getW(), t.getH(), null);
            } else {
                // Fallback falls kein Bild existiert
                g.setColor(new Color(80, 80, 80));
                g.fillRect(t.getX() - camX, t.getY(), t.getW(), t.getH());
            }
        }

        // Flagge
        g.setColor(new Color(50, 50, 50));
        g.fillRect(endX - camX - 10, endY + TILE_SIZE - 20, 60, 20);

        g.setColor(new Color(200, 200, 200));
        g.fillRect(endX - camX, endY + TILE_SIZE - 20 - 150, 5, 150);

        g.setColor(Color.RED);
        g.fillRect(endX - camX + 5, endY + TILE_SIZE - 20 - 150, 40, 25);
        g.setColor(Color.BLACK);
        g.drawRect(endX - camX + 5, endY + TILE_SIZE - 20 - 150, 40, 25);
    }

    public ArrayList<Tile> getSolidTiles() {
        return solidTiles;
    }

    public int[][] getEnemyPositions() {
        return enemyPositions;
    }

    public int getWidth() {
        return width;
    }

    public int getHeight() {
        return height;
    }
}

In [ ]:
import javax.imageio.ImageIO;
import java.awt.*;
import java.awt.image.BufferedImage;
import java.io.File;

public class Player {
    private final int w = 32, h = 48;
    private final Level level;
    private final double GRAVITY = 0.6;
    private final double MOVE_SPEED = 3.2;
    private final double JUMP_SPEED = -12.5;
    private final double MAX_FALL = 14.0;
    private final double COYOTE_TIME = 0.12;
    private final double JUMP_BUFFER_TIME = 0.12;
    private final double startX, startY;

    private double x, y;
    private double vx, vy;
    private boolean onGround = false;
    private double coyoteTimer = 0.0;
    private double jumpBufferTimer = 0.0;

    // ------- SPRITE -------
    private BufferedImage sprite = null;
    private boolean flipX = false;

    public Player(double startX, double startY, Level level) {
        this.x = startX;
        this.y = startY;
        this.startX = startX;
        this.startY = startY;
        this.level = level;
    }

    // Bild direkt aus Datei laden
    public void loadSprite(String path) {
        try {
            sprite = ImageIO.read(new File(path));
        } catch (Exception e) {
            System.err.println("Sprite konnte nicht geladen werden: " + path);
        }
    }

    public void update(double dt) {
        if (coyoteTimer > 0) coyoteTimer -= dt;
        if (jumpBufferTimer > 0) jumpBufferTimer -= dt;

        vy += GRAVITY;
        if (vy > MAX_FALL) vy = MAX_FALL;

        // --- horizontale Bewegung ---
        x += vx;
        Rectangle hb = getBounds();
        for (Tile t : level.getSolidTiles()) {
            if (hb.intersects(t.getRect())) {
                if (vx > 0) x = t.getX() - w;
                else if (vx < 0) x = t.getX() + t.getW();
                vx = 0;
                hb = getBounds();
            }
        }

        // --- vertikale Bewegung ---
        y += vy;
        onGround = false;
        for (Tile t : level.getSolidTiles()) {
            Rectangle tileRect = t.getRect();
            if (getBounds().intersects(tileRect)) {
                if (vy > 0) {
                    y = tileRect.y - h;
                    vy = 0;
                    onGround = true;
                    coyoteTimer = COYOTE_TIME;
                } else if (vy < 0) {
                    y = tileRect.y + tileRect.height;
                    vy = 0;
                }
            }
        }

        // --- Sprungpuffer ---
        if (jumpBufferTimer > 0 && (onGround || coyoteTimer > 0)) {
            doJump();
            jumpBufferTimer = 0;
            coyoteTimer = 0;
        }

        // --- Levelgrenzen ---
        if (x < 0) x = 0;
        if (x + w > level.getWidth()) x = level.getWidth() - w;
        if (y > level.getHeight() + 300) {
            respawn();
            GamePanel.subLive();
        }
    }

    // -------- Bewegung ----------
    public void moveLeft() {
        vx = -MOVE_SPEED;
        flipX = true;   // Sprite spiegeln
    }

    public void moveRight() {
        vx = MOVE_SPEED;
        flipX = false;  // Normal
    }

    public void stopHorizontal() {
        vx = 0;
    }

    public void pressJump() {
        jumpBufferTimer = JUMP_BUFFER_TIME;
    }

    public void releaseJump() {
        if (vy < 0) vy *= 0.6;
    }

    private void doJump() {
        vy = JUMP_SPEED;
        onGround = false;
    }

    public void bounceAfterStomp() {
        vy = JUMP_SPEED / 2;
    }

    public void respawn() {
        x = startX;
        y = startY;
        vx = 0;
        vy = 0;
    }

    public boolean isFalling() {
        return vy > 0;
    }

    // -------- Bounds / Position ----------
    public Rectangle getBounds() {
        return new Rectangle((int) Math.round(x), (int) Math.round(y), w, h);
    }

    public int getY() {
        return (int) Math.round(y);
    }

    public int getX() {
        return (int) Math.round(x);
    }

    // -------- Draw ----------
    public void draw(Graphics2D g, int camX) {
        int drawX = getX() - camX;
        int drawY = getY();

        if (sprite != null) {
            // Sprite zeichnen (links/rechts gespiegelt)
            if (!flipX) {
                g.drawImage(sprite, drawX, drawY, w, h, null);
            } else {
                g.drawImage(sprite, drawX + w, drawY, drawX, drawY + h,   // gespiegeltes Ziel
                        0, 0, sprite.getWidth(), sprite.getHeight(), null);
            }
        } else {
            // Fallback: alte Player-Zeichnung
            g.setColor(new Color(200, 30, 30));
            g.fillRoundRect(drawX, drawY, w, h, 6, 6);

            g.setColor(Color.WHITE);
            g.fillOval(drawX + 8, drawY + 8, 8, 8);

            g.setColor(Color.BLACK);
            g.fillOval(drawX + 10, drawY + 10, 3, 3);
        }
    }
}

In [ ]:
import java.awt.*;
import java.awt.image.BufferedImage;

public class Tile {
    private final int x;
    private final int y;
    private final int w;
    private final int h;
    private BufferedImage texture;

    public Tile(int x, int y, int w, int h,  BufferedImage texture) {
        this.x = x;
        this.y = y;
        this.w = w;
        this.h = h;
        this.texture = texture;
    }

    public void draw(Graphics2D g, int camX) {
        if (texture != null) {
            g.drawImage(texture, x - camX, y, w, h, null);
        } else {
            g.setColor(new Color(150, 75, 0));
            g.fillRect(x - camX, y, w, h);
        }
    }

    public Rectangle getRect() {
        return new Rectangle(x, y, w, h);
    }

    public int getX() {
        return x;
    }

    public int getY() {
        return y;
    }

    public int getW() {
        return w;
    }

    public int getH() {
        return h;
    }
}

Die Bilder für die einzelnen Assets haben wir selbständig gepixelt, da die Bildgenerierung nicht zuverlässig funktioniert hat. Die generierten Grafiken entsprachen nicht den vorgegebenen Maßen, wodurch sie beim Einfügen stark verzerrt dargestellt wurden.

Als nächste Erweiterung haben wir einen Menü-Screen implementiert. Die Entwicklung mit Unterstützung eines LLM funktionierte jedoch nur teilweise: Das Menü ließ sich in einigen Fällen nicht aus dem Spiel heraus erneut öffnen oder war zwar funktional, wurde aber grafisch nicht korrekt aktualisiert. Dadurch musste man teilweise raten, wo sich die einzelnen Buttons befinden.

Zu den verwendeten Prompts gehörten unter anderem:

„Erstelle einen Menü-Screen. In diesem soll man das Spiel starten, die Steuerung anzeigen lassen, einzelne Level auswählen, um nicht immer von vorne anfangen zu müssen, und das Spiel beenden können.“

„Das Menü soll auch mit ESC aus dem Level heraus geöffnet werden können.“

„Problem: Man kann aus dem Level zwar das Menü aufrufen und die Buttons betätigen, die Anzeige wird jedoch nicht richtig aktualisiert. Man sieht weiterhin nur das Level, und die Menüansicht wird nicht angezeigt.“

Zusätzlich haben wir eine Skalierungsfunktion eingebaut, mit der man das Spiel größer oder kleiner darstellen kann, und die Projektstruktur mithilfe von Packages übersichtlicher gestaltet.
Ebenfalls ergänzt wurden die Level 5–8, die wir nach dem Vorbild der ersten Level erstellt haben. Dafür wurde unter anderem die Klasse MovingPlatform entwickelt sowie eine Unterscheidung zwischen beweglichen und unbeweglichen Gegnern eingeführt.

Darüber hinaus haben wir mehrere Quality-of-Life-Verbesserungen vorgenommen, darunter kleinere Überarbeitungen in den Leveln, Anpassungen am Score-System sowie Verbesserungen an der Kollisionslogik.

Durch die vorgenommenen Erweiterungen und Anpassungen entstand schließlich die folgende Projektstruktur:

src/
│
├─ entities/        # Spielfiguren & Gegner
│  ├─ Enemy
│  ├─ MovingEnemy
│  └─ Player
│
├─ levels/          # Levelarchitektur & Plattformen
│  ├─ Level
│  ├─ MovingPlatform
│  └─ Tile
│
├─ main/            # Hauptlogik & Programmstart
│  ├─ Game
│  ├─ GamePanel
│  ├─ Main
│  └─ Storage
│
├─ ui/              # Menü und Bildschirmverwaltung
│  ├─ LoadingScreens
│  └─ MenuManager
│
└─ utils/           # Hilfsklassen
   ├─ GameState
   └─ Zoom

res/                # Ressourcen (Grafiken usw.)


Der neue Code sah nach allen Bugfixes entsprechend so aus:

In [ ]:
package entities;

import levels.Level;
import levels.Tile;
import utils.Zoom;

import javax.imageio.ImageIO;
import java.awt.*;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;

public class Enemy {
    public static final int ENEMY_HEIGHT = (int) (32 * Zoom.SCALE);
    public final int ENEMY_WIDTH = (int) (32 * Zoom.SCALE);
    protected Level level;
    protected double x;
    protected double y;
    protected BufferedImage sprite = null;
    protected boolean flipX = false;

    public Enemy(double x, double y, Level level) {
        this.x = x;
        this.y = y;
        this.level = level;
        try {
            sprite = ImageIO.read(new File("res/Geist_V3.png"));
        } catch (IOException e) {
            System.err.println("entities.MovingEnemy sprite loading failed.");
        }
    }

    public void update() {

        // ==========================
        // 1) GRAVITY (immer zuerst!)
        // ==========================
        double nextY = y + 4;  // Fallgeschwindigkeit

        boolean onGround = false;
        for (Tile t : level.getSolidTiles()) {
            Rectangle test = new Rectangle((int) x, (int) nextY, ENEMY_WIDTH, ENEMY_HEIGHT);
            if (test.intersects(t.getRect())) {
                // auf Boden setzen
                y = t.getY() - ENEMY_HEIGHT;
                onGround = true;
                break;
            }
        }

        if (!onGround) {
            // frei fallender Gegner → FALLEN
            y = nextY;
        }
    }


    public Rectangle getBounds() {
        return new Rectangle((int) Math.round(x), (int) Math.round(y), ENEMY_WIDTH, ENEMY_HEIGHT);
    }

    public int getY() {
        return (int) Math.round(y);
    }

    public void draw(Graphics2D g, int camX) {
        int dx = (int) Math.round(x) - camX;
        int dy = (int) Math.round(y);

        if (sprite != null) {
            if (!flipX) {
                g.drawImage(sprite, dx, dy, ENEMY_WIDTH, ENEMY_HEIGHT, null);
            } else {
                g.drawImage(sprite, dx + ENEMY_WIDTH, dy, dx, dy + ENEMY_HEIGHT, 0, 0, sprite.getWidth(), sprite.getHeight(), null);
            }
        } else {
            g.setColor(new Color(255, 140, 0));
            g.fillOval(dx, dy, ENEMY_WIDTH, ENEMY_HEIGHT);
            g.setColor(Color.BLACK);
            g.drawOval(dx, dy, ENEMY_WIDTH, ENEMY_HEIGHT);
        }
    }
}

In [ ]:
package entities;

import levels.Level;
import levels.Tile;
import utils.Zoom;

import java.awt.*;

public class MovingEnemy extends Enemy {

    private double vx = 1.2 * Zoom.SCALE;

    public MovingEnemy(double x, double y, Level level) {
        super(x, y, level);

        this.x = x;
        this.y = y;
        this.level = level;
    }

    @Override
    public void update() {
        super.update();

        // =========================================
        // 2) ANTI-FALL-KANTEN-ERKENNUNG (nur wenn Boden!)
        // =========================================
        Rectangle frontFoot = new Rectangle(
                (int)(x + (vx > 0 ? ENEMY_WIDTH : -4)),
                (int)(y + ENEMY_HEIGHT),
                4,           // Breite der Abfrage
                4            // Höhe der Abfrage
        );


        boolean groundAhead = false;
        for (Tile t : level.getSolidTiles()) {
            if (frontFoot.intersects(t.getRect())) {
                groundAhead = true;
                break;
            }
        }


        if (!groundAhead) {
            // keine Kante → umdrehen
            vx = -vx;
            flipX = !flipX;
            return;
        }

        // ==========================
        // 3) HORIZONTALE BEWEGUNG
        // ==========================
        int sensorX = (int)(x + (vx > 0 ? ENEMY_WIDTH + 1 : -5));
        int sensorY = (int)y + 4;        // kleine Höhe von oben weg, nicht von der Mitte
        int sensorH = ENEMY_HEIGHT - 8;  // nicht über den Boden ziehen

        Rectangle sideSensor = new Rectangle(sensorX, sensorY, 4, sensorH);

        boolean wallAhead = false;
        for (Tile t : level.getSolidTiles()) {
            if (sideSensor.intersects(t.getRect())) {
                wallAhead = true;
                break;
            }
        }

        if (wallAhead) {
            // Wand → wenden
            vx = -vx;
            flipX = !flipX;
            return;
        }

        // Wenn alles ok → bewegen
        x += vx;
    }
}

In [ ]:
package entities;

import levels.Level;
import levels.MovingPlatform;
import levels.Tile;
import main.GamePanel;
import utils.GameState;
import utils.Zoom;

import javax.imageio.ImageIO;
import java.awt.*;
import java.awt.image.BufferedImage;
import java.io.File;

public class Player {
    public static final int PLAYER_WIDTH = (int) (32 * Zoom.SCALE);
    public static final int PLAYER_HEIGHT = (int) (48 * Zoom.SCALE);
    private static final int HORIZONTAL_COLLISION_PADDING = (int) (1 * Zoom.SCALE);

    private final Level level;
    private final double MOVE_SPEED = 2.5 * Zoom.SCALE;
    private final double JUMP_SPEED = -11 * Zoom.SCALE;

    private final double startX, startY;
    private double x, y;
    private double vx, vy;
    private boolean onGround = false;

    private double coyoteTimer = 0.0;
    private double jumpBufferTimer = 0.0;

    private BufferedImage sprite = null;
    private boolean flipX = false;

    private MovingPlatform currentPlatform = null;
    private double platformPrevX = 0;
    private double platformPrevY = 0;

    public Player(double startX, double startY, Level level) {
        this.x = startX;
        this.y = startY;
        this.startX = startX;
        this.startY = startY;
        this.level = level;
    }

    public void loadSprite(String path) {
        try {
            sprite = ImageIO.read(new File(path));
        } catch (Exception e) {
            System.err.println("Sprite konnte nicht geladen werden: " + path);
        }
    }

    public boolean isFalling() {
        return vy > 0 && !onGround;
    }

    public void update(double dt) {
        double GRAVITY = 0.6 * Zoom.SCALE;
        double MAX_FALL = 14.0 * Zoom.SCALE;
        double COYOTE_TIME = 0.12;

        if (coyoteTimer > 0) coyoteTimer -= dt;
        if (jumpBufferTimer > 0) jumpBufferTimer -= dt;

        // Schwerkraft
        vy += GRAVITY * dt * 60;
        if (vy > MAX_FALL) vy = MAX_FALL;

        // Horizontale Bewegung + Kollisionsabfrage
        x += vx * dt * 60;
        Rectangle hb = getBounds();
        for (Tile t : level.getSolidTiles()) {
            if (hb.intersects(t.getRect())) {
                if (vx > 0) x = t.getX() - PLAYER_WIDTH;
                else if (vx < 0) x = t.getX() + t.getW();
                vx = 0;
                hb = getBounds();
            }
        }

        // Vertikale Bewegung + Kollisionsabfrage
        y += vy * dt * 60;
        onGround = false;

        Rectangle verticalBounds = new Rectangle(getX() + HORIZONTAL_COLLISION_PADDING, getY(), PLAYER_WIDTH - (2 * HORIZONTAL_COLLISION_PADDING), PLAYER_HEIGHT);

        MovingPlatform newPlatform = null;

        for (Tile t : level.getSolidTiles()) {
            Rectangle tileRect = t.getRect();
            if (verticalBounds.intersects(tileRect)) {
                if (vy > 0) { // Landen
                    y = tileRect.y - PLAYER_HEIGHT;
                    vy = 0;
                    onGround = true;
                    coyoteTimer = COYOTE_TIME;

                    if (t instanceof MovingPlatform mp) {
                        newPlatform = mp;
                    }
                } else if (vy < 0) { // Kopfstoß
                    y = tileRect.y + tileRect.height;
                    vy = 0;
                }
            }
        }

        // Plattformbewegung anwenden
        if (newPlatform != null) {
            if (currentPlatform == newPlatform) {
                // Unterschied zur letzten Position berechnen
                double dx = newPlatform.getX() - platformPrevX;
                double dy = newPlatform.getY() - platformPrevY;
                x += dx;
                y += dy;
            }
            platformPrevX = newPlatform.getX();
            platformPrevY = newPlatform.getY();
        }
        currentPlatform = newPlatform;

        // Jump-buffering + Coyote Time
        if (jumpBufferTimer > 0 && (onGround || coyoteTimer > 0)) {
            doJump();
            jumpBufferTimer = 0;
            coyoteTimer = 0;
        }

        // levels.Level-Grenzen
        if (x < 0) x = 0;
        if (x + PLAYER_WIDTH > level.getWidth()) x = level.getWidth() - PLAYER_WIDTH;

        // Unter den Boden fallen
        if (y > level.getHeight() + 300) {
            GamePanel.subLive();
            if (GamePanel.lives <= 0) {
                GamePanel.state = GameState.GAME_OVER_SCREEN;
            } else {
                respawn();
            }
        }
    }

    // Steuerung
    public void moveLeft() {
        vx = -MOVE_SPEED;
        flipX = true;
    }

    public void moveRight() {
        vx = MOVE_SPEED;
        flipX = false;
    }

    public void stopHorizontal() {
        vx = 0;
    }

    public void pressJump() {
        jumpBufferTimer = 0.12;
    }

    public void releaseJump() {
        if (vy < 0) vy *= 0.6;
    }

    private void doJump() {
        vy = JUMP_SPEED;
        onGround = false;
        currentPlatform = null;
    }

    public void bounceAfterStomp() {
        vy = JUMP_SPEED / 2;
    }

    public void respawn() {
        x = startX;
        y = startY;
        vx = 0;
        vy = 0;
        currentPlatform = null;
        platformPrevX = 0;
        platformPrevY = 0;
    }

    // BOUNDS + POSITION
    public Rectangle getBounds() {
        return new Rectangle((int) Math.round(x), (int) Math.round(y), PLAYER_WIDTH, PLAYER_HEIGHT);
    }

    public int getX() {
        return (int) Math.round(x);
    }

    public int getY() {
        return (int) Math.round(y);
    }

    // Zeichnen
    public void draw(Graphics2D g, int camX) {
        int drawX = getX() - camX;
        int drawY = getY();

        if (sprite != null) {
            if (!flipX) {
                g.drawImage(sprite, drawX, drawY, PLAYER_WIDTH, PLAYER_HEIGHT, null);
            } else {
                g.drawImage(sprite, drawX + PLAYER_WIDTH, drawY, drawX, drawY + PLAYER_HEIGHT, 0, 0, sprite.getWidth(), sprite.getHeight(), null);
            }
        } else {
            g.setColor(new Color(200, 30, 30));
            g.fillRoundRect(drawX, drawY, PLAYER_WIDTH, PLAYER_HEIGHT, 6, 6);
            g.setColor(Color.WHITE);
            g.fillOval(drawX + 8, drawY + 8, 8, 8);
            g.setColor(Color.BLACK);
            g.fillOval(drawX + 10, drawY + 10, 3, 3);
        }
    }
}

In [ ]:
package levels;

import entities.Player;
import main.GamePanel;
import utils.Zoom;

import javax.imageio.ImageIO;
import java.awt.*;
import java.awt.image.BufferedImage;
import java.io.File;
import java.util.ArrayList;

public class Level {
    private static final int BASE_TILE_SIZE = 32;
    // TILE_SIZE wird korrekt mit dem Skalierungsfaktor multipliziert
    public static final int TILE_SIZE = (int) (BASE_TILE_SIZE * Zoom.SCALE);
    public static final int groundY = GamePanel.HEIGHT - TILE_SIZE;
    // Annahme: PLAYER_HEIGHT ist in entities.Player.java korrekt skaliert
    private static final int PLAYER_HEIGHT = Player.PLAYER_HEIGHT;
    private static BufferedImage groundTexture;
    private static BufferedImage blockTexture;
    private static BufferedImage background;

    // Ressourcen werden nur einmal beim Start geladen (sauberer)
    static {
        try {
            groundTexture = ImageIO.read(new File("res/ground_V2.png"));
            blockTexture = ImageIO.read(new File("res/block_V3.png"));
            background = ImageIO.read(new File("res/background_V2.png"));
        } catch (Exception e) {
            System.err.println("Fehler beim Laden der Level-Texturen.");
        }
    }

    private final int width;
    private final int height;
    private ArrayList<Tile> solidTiles = new ArrayList<>();
    private int[][] enemyPositions;
    private int endX, endY;

    // Konstruktor ist jetzt nur für die Level-Dimensionen zuständig
    public Level(int width, int height) {
        this.width = width;
        this.height = height;
    }

    public static Level createSampleLevel(int index) {
        // Level-Dimensionen skalieren
        Level level = new Level((int) (3000 * Zoom.SCALE), (int) (600 * Zoom.SCALE));
        ArrayList<Tile> tiles = new ArrayList<>();

        int spawnY = groundY - TILE_SIZE;

        // Grundboden mit kleinen Lücken
        for (int x = 0; x < 100; x++) {
            //if (x % 8 == 4 && index > 0) continue;
            tiles.add(new Tile(x * TILE_SIZE, groundY, TILE_SIZE, TILE_SIZE, groundTexture));
        }

        // -----------------------
        // LEVEL 1 – einfaches Springen & Bewegung
        // -----------------------
        if (index == 0) {
            tiles.add(new Tile(8 * TILE_SIZE, groundY - TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(10 * TILE_SIZE, groundY - TILE_SIZE - PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(12 * TILE_SIZE, groundY - TILE_SIZE - 2 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));

            // neue Sektion 2
            tiles.add(new Tile(19 * TILE_SIZE, groundY - TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(21 * TILE_SIZE, groundY - TILE_SIZE - PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(23 * TILE_SIZE, groundY - TILE_SIZE - 2 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(25 * TILE_SIZE, groundY - TILE_SIZE - 3 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));

            // Endzone mit leichter Rampe: 10 Pixel Steigung/Höhe skaliert
            int rampStep = (int) (10 * Zoom.SCALE);
            for (int x = 38; x < 45; x++) {
                tiles.add(new Tile(x * TILE_SIZE, groundY - (x - 38) * rampStep - rampStep, TILE_SIZE, rampStep, blockTexture));
            }

            level.enemyPositions = new int[][]{{14 * TILE_SIZE, spawnY, 0}, {26 * TILE_SIZE, spawnY, 0}, {32 * TILE_SIZE, spawnY, 0}};

            level.endX = 48 * TILE_SIZE;
            level.endY = groundY - TILE_SIZE;
        }

        // -----------------------
        // LEVEL 2 – Doppelsprünge, Hindernisse, kleine Gruben
        // -----------------------
        else if (index == 1) {
            tiles.add(new Tile(8 * TILE_SIZE, groundY - TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(10 * TILE_SIZE, groundY - TILE_SIZE - PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(12 * TILE_SIZE, groundY - TILE_SIZE - 2 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(15 * TILE_SIZE, groundY - TILE_SIZE - 2 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(18 * TILE_SIZE, groundY - TILE_SIZE - 3 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));

            // zweite Hälfte mit erhöhten Plattformen: 50 Pixel Offset skaliert
            int platformOffset = (int) (50 * Zoom.SCALE);
            for (int i = 0; i < 5; i++) {
                tiles.add(new Tile((int) (24 * TILE_SIZE + i * TILE_SIZE * 1.4), groundY - TILE_SIZE - PLAYER_HEIGHT - (i % 2) * platformOffset, TILE_SIZE, TILE_SIZE, blockTexture));
            }

            tiles.add(new Tile(33 * TILE_SIZE, groundY - 2 * TILE_SIZE - 2 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(35 * TILE_SIZE, groundY - 2 * TILE_SIZE - PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(38 * TILE_SIZE, groundY - 2 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(40 * TILE_SIZE, groundY - TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));

            level.enemyPositions = new int[][]{{12 * TILE_SIZE, spawnY, 0}, {17 * TILE_SIZE, spawnY, 0}, {26 * TILE_SIZE, spawnY, 1}, {34 * TILE_SIZE, spawnY, 0}};

            level.endX = 48 * TILE_SIZE;
            level.endY = groundY - TILE_SIZE;
        }

        // -----------------------
        // LEVEL 3 – Gegnerpfade & anspruchsvollere Sprünge
        // -----------------------
        else if (index == 2) {
            // ... (Hier sind keine weiteren Pixel-Offsets zu skalieren, da nur TILE_SIZE und PLAYER_HEIGHT verwendet werden, die bereits skaliert sind)

            tiles.add(new Tile(8 * TILE_SIZE, groundY - TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(10 * TILE_SIZE, groundY - TILE_SIZE - PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(12 * TILE_SIZE, groundY - TILE_SIZE - 2 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));

            // tiefe Lücke + Plattform darüber
            for (int x = 13; x < 17; x++) {
                int finalX = x;
                tiles.removeIf(t -> t.getX() / TILE_SIZE == finalX);
            }
            tiles.add(new Tile(14 * TILE_SIZE, groundY - TILE_SIZE - 3 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(16 * TILE_SIZE, groundY - TILE_SIZE - 4 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(18 * TILE_SIZE, groundY - TILE_SIZE - 3 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));

            // zweite Sektion – gestaffelte Höhen
            tiles.add(new Tile(24 * TILE_SIZE, groundY - TILE_SIZE - PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(26 * TILE_SIZE, groundY - TILE_SIZE - 2 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(28 * TILE_SIZE, groundY - TILE_SIZE - 3 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(30 * TILE_SIZE, groundY - TILE_SIZE - 2 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(32 * TILE_SIZE, groundY - TILE_SIZE - PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(49 * TILE_SIZE, groundY - TILE_SIZE, TILE_SIZE, TILE_SIZE, null));

            level.enemyPositions = new int[][]{{11 * TILE_SIZE, spawnY, 1}, {22 * TILE_SIZE, spawnY, 0}, {29 * TILE_SIZE, spawnY, 0}, {38 * TILE_SIZE, spawnY, 1}};

            level.endX = 48 * TILE_SIZE;
            level.endY = groundY - TILE_SIZE;
        }

        // -----------------------
        // LEVEL 4 – Finale: mehr vertikale & lange Sprungpassagen
        // -----------------------
        else if (index == 3) {
            tiles.add(new Tile(8 * TILE_SIZE, groundY - TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(10 * TILE_SIZE, groundY - TILE_SIZE - PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(12 * TILE_SIZE, groundY - TILE_SIZE - 2 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(15 * TILE_SIZE, groundY - TILE_SIZE - 3 * PLAYER_HEIGHT, TILE_SIZE, TILE_SIZE, blockTexture));

            // mittlere Plattform-Linie: 50 Pixel Offset skaliert
            int platformOffset = (int) (50 * Zoom.SCALE);
            for (int i = 0; i < 7; i++) {
                tiles.add(new Tile(20 * TILE_SIZE + i * 2 * TILE_SIZE, groundY - TILE_SIZE - 2 * PLAYER_HEIGHT - (i % 2) * platformOffset, TILE_SIZE, TILE_SIZE, blockTexture));
            }

            // finale Rampe hoch zur Flagge: TILE_SIZE ist bereits skaliert, hier keine weitere Skalierung nötig
            for (int i = 0; i < 6; i++) {
                tiles.add(new Tile(37 * TILE_SIZE + i * TILE_SIZE, (int) (groundY - TILE_SIZE - i * 0.5 * TILE_SIZE), TILE_SIZE, TILE_SIZE, blockTexture));
            }

            tiles.add(new Tile(46 * TILE_SIZE, groundY - 3 * PLAYER_HEIGHT + 2 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));

            level.enemyPositions = new int[][]{{13 * TILE_SIZE, spawnY, 0}, {22 * TILE_SIZE, spawnY, 1}, {28 * TILE_SIZE, spawnY, 1}, {34 * TILE_SIZE, spawnY, 1}, {43 * TILE_SIZE, spawnY, 0}};

            level.endX = 46 * TILE_SIZE;
            level.endY = groundY - 3 * PLAYER_HEIGHT + TILE_SIZE;
        }

        // -------
        // LEVEL 5
        // -------
        else if (index == 4) {
            // 1) Erzeuge längeren Boden, dann große Lücke (Canyon)
            // entferne Boden von tx=36..68 (Canyon)
            tiles.removeIf(t -> {
                int tx = t.getX() / TILE_SIZE;
                return tx >= 36 && tx <= 68;
            });

            // 2) Dichte Anordnung links vor dem Canyon (Stufen/Plattformen)
            tiles.add(new Tile(10 * TILE_SIZE, groundY - TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(12 * TILE_SIZE, groundY - 2 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(14 * TILE_SIZE, groundY - 3 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(16 * TILE_SIZE, groundY - 2 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(18 * TILE_SIZE, groundY - TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));

            // 3) horizontale Movers (Kettenbrücke über Canyon)
            // Plattform A: bewegt sich kurz hin und her; Spieler springt auf sie, sie fährt zu Insel
            tiles.add(new MovingPlatform(36 * TILE_SIZE, groundY - 2 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 42 * TILE_SIZE, groundY - 2 * TILE_SIZE, 1.8 * Zoom.SCALE, blockTexture));

            // Plattform B: startet auf der Insel-Seite und pendelt; chain notwendig
            tiles.add(new MovingPlatform(50 * TILE_SIZE, groundY - 2 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 44 * TILE_SIZE, groundY - 2 * TILE_SIZE, 1.8 * Zoom.SCALE, blockTexture));

            // 4) Insel-Anordnung nach Canyon (mehrere Stufen, hoch zum Turm)
            tiles.add(new Tile(52 * TILE_SIZE, groundY - TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(54 * TILE_SIZE, groundY - 2 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(56 * TILE_SIZE, groundY - 3 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(58 * TILE_SIZE, groundY - 4 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));

            // 5) Zweiter mover, um auf die hohe Turmplattform zu kommen
            tiles.add(new MovingPlatform(61 * TILE_SIZE, groundY - 4 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 74 * TILE_SIZE, groundY - 4 * TILE_SIZE, 1.4 * Zoom.SCALE, blockTexture));

            // 6) Turmaufbau zur Flagge (Höhe: 4 tiles)
            for (int h = 0; h < 5; h++) {
                tiles.add(new Tile(78 * TILE_SIZE, groundY - (h + 1) * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            }
            //tiles.add(new levels.Tile(80 * TILE_SIZE, groundY - 6 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture)); // Flag base

            level.enemyPositions = new int[][]{{13 * TILE_SIZE, spawnY, 1}, {34 * TILE_SIZE, spawnY, 1}};
            level.endX = 80 * TILE_SIZE;
            level.endY = groundY - TILE_SIZE;
        }

        // -------
        // LEVEL 6
        // -------
        else if (index == 5) {
            // 1) Definiere mehrere Gaps (unten durchlaufen unmöglich)
            int[][] gaps = new int[][]{{8, 14},   // early gap
                    {22, 28},  // mid gap
                    {38, 46},  // long mid gap
                    {56, 64}   // late gap before ascent
            };
            for (int[] g : gaps) {
                int s = g[0], e = g[1];
                tiles.removeIf(t -> {
                    int tx = t.getX() / TILE_SIZE;
                    return tx >= s && tx <= e;
                });
            }

            // 2) Start-Platforms (präzise)
            tiles.add(new Tile(6 * TILE_SIZE, groundY - TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(7 * TILE_SIZE, groundY - 2 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));

            // 3) Kette von horizontalen moving platforms über die ersten Gaps
            tiles.add(new MovingPlatform(10 * TILE_SIZE, groundY - 2 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 13 * TILE_SIZE, groundY - 2 * TILE_SIZE, 1.6 * Zoom.SCALE, blockTexture));
            tiles.add(new Tile(21 * TILE_SIZE, groundY - TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new MovingPlatform(24 * TILE_SIZE, groundY - 3 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 30 * TILE_SIZE, groundY - 3 * TILE_SIZE, 1.8 * Zoom.SCALE, blockTexture));
            tiles.add(new MovingPlatform(40 * TILE_SIZE, groundY - 2 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 45 * TILE_SIZE, groundY - 2 * TILE_SIZE, 2.0 * Zoom.SCALE, blockTexture));

            // 4) Zwischenplattformen + kleine Türme (erhöhte Plateaus)
            tiles.add(new Tile(50 * TILE_SIZE, groundY - TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(52 * TILE_SIZE, groundY - 2 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(54 * TILE_SIZE, groundY - 4 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));

            // 5) Langer „swing“ mover zu finaler Ascent
            tiles.add(new MovingPlatform(58 * TILE_SIZE, groundY - 4 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 67 * TILE_SIZE, groundY - 4 * TILE_SIZE, 1.3 * Zoom.SCALE, blockTexture));

            // 6) Finale Stufen und Flagge (hoch)
            tiles.add(new Tile(72 * TILE_SIZE, groundY - TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(72 * TILE_SIZE, groundY - 2 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(74 * TILE_SIZE, groundY - 3 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));

            tiles.add(new Tile(76 * TILE_SIZE, groundY - 4 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));

            level.enemyPositions = new int[][]{{15 * TILE_SIZE, spawnY, 1}, {30 * TILE_SIZE, spawnY, 1}, {47 * TILE_SIZE, spawnY, 1}};
            level.endX = 76 * TILE_SIZE;
            level.endY = groundY - 5 * TILE_SIZE;
        }


        // -------
        // LEVEL 7
        // -------
        else if (index == 6) {
            // 1) Großer zentraler „Trough“: entferne Boden großflächig, aber lasse sparsely stepping tiles
            tiles.removeIf(t -> {
                int tx = t.getX() / TILE_SIZE;
                return tx >= 6 && tx <= 110 && (tx % 2 == 0); // viele Lücken, aber ein paar stepping stones
            });

            // 2) Serie horizontaler movers auf progressively higher rows
            tiles.add(new MovingPlatform(12 * TILE_SIZE, groundY - 3 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 20 * TILE_SIZE, groundY - 3 * TILE_SIZE, 2.0 * Zoom.SCALE, blockTexture));
            tiles.add(new MovingPlatform(30 * TILE_SIZE, groundY - 5 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 22 * TILE_SIZE, groundY - 5 * TILE_SIZE, 2.0 * Zoom.SCALE, blockTexture));
            tiles.add(new MovingPlatform(32 * TILE_SIZE, groundY - 7 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 40 * TILE_SIZE, groundY - 7 * TILE_SIZE, 2.0 * Zoom.SCALE, blockTexture));

            // 3) stacked vertical movers that form the ascent to the fortress roof
            tiles.add(new MovingPlatform(44 * TILE_SIZE, groundY - 3 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 44 * TILE_SIZE, groundY - 10 * TILE_SIZE, Zoom.SCALE, blockTexture));
            tiles.add(new MovingPlatform(48 * TILE_SIZE, groundY - 5 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 48 * TILE_SIZE, groundY - 12 * TILE_SIZE, 1.2 * Zoom.SCALE, blockTexture));
            tiles.add(new MovingPlatform(52 * TILE_SIZE, groundY - 6 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 52 * TILE_SIZE, groundY - 14 * TILE_SIZE, 1.1 * Zoom.SCALE, blockTexture));

            // 4) Dense stepping stones on the fortress slopes (increase level density)
            for (int i = 0; i < 5; i++) {
                if (i == 1) continue;
                tiles.add(new Tile((47 + i) * TILE_SIZE, groundY - ((i % 5) + 4) * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            }

            // 5) Top of fortress and flag
            tiles.add(new Tile(55 * TILE_SIZE, groundY - 15 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(56 * TILE_SIZE, groundY - 15 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture)); // flag base

            level.enemyPositions = new int[][]{{13 * TILE_SIZE, groundY - 2 * TILE_SIZE, 0}, {49 * TILE_SIZE, groundY - 6 * TILE_SIZE, 0}};
            level.endX = 56 * TILE_SIZE;
            level.endY = groundY - 16 * TILE_SIZE;
        }

        // -------
        // Level 8
        // -------
        else if (index == 7) {
            // 1) Entferne Boden in einem breiten mittleren Bereich (unpassierbar)
            tiles.removeIf(t -> {
                int tx = t.getX() / TILE_SIZE;
                return tx >= 14 && tx <= 38;
            });

            // 2) Basis-Cluster links
            tiles.add(new Tile(10 * TILE_SIZE, groundY - TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(11 * TILE_SIZE, groundY - 2 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));

            tiles.add(new MovingPlatform(13 * TILE_SIZE, groundY - 3 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 18 * TILE_SIZE, groundY - 3 * TILE_SIZE, 1.6 * Zoom.SCALE, blockTexture));

            // 3) Vertikale Fahrstühle (essenziell) — zwei Etagen hoch
            tiles.add(new MovingPlatform(20 * TILE_SIZE, groundY - 2 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 20 * TILE_SIZE, groundY - 8 * TILE_SIZE, Zoom.SCALE, blockTexture)); // elevator A
            tiles.add(new MovingPlatform(24 * TILE_SIZE, groundY - 3 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 24 * TILE_SIZE, groundY - 11 * TILE_SIZE, 1.4 * Zoom.SCALE, blockTexture)); // elevator B

            // 4) Horizontal movers zwischen Etagen (Chain required)
            tiles.add(new MovingPlatform(21 * TILE_SIZE, groundY - 7 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 27 * TILE_SIZE, groundY - 7 * TILE_SIZE, 1.4 * Zoom.SCALE, blockTexture));
            tiles.add(new MovingPlatform(27 * TILE_SIZE, groundY - 8 * TILE_SIZE, TILE_SIZE, TILE_SIZE, 21 * TILE_SIZE, groundY - 8 * TILE_SIZE, 1.4 * Zoom.SCALE, blockTexture));

            // 5) Obere Plattformen + finale Flagge (hoch)
            tiles.add(new Tile(27 * TILE_SIZE, groundY - 12 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture));
            tiles.add(new Tile(27 * TILE_SIZE + TILE_SIZE, groundY - 12 * TILE_SIZE, TILE_SIZE, TILE_SIZE, blockTexture)); // flag base

            level.enemyPositions = new int[][]{{11 * TILE_SIZE, groundY - 4 * TILE_SIZE, 0}, {24 * TILE_SIZE, groundY - 4 * TILE_SIZE, 0}};
            level.endX = 27 * TILE_SIZE + TILE_SIZE;
            level.endY = groundY - 13 * TILE_SIZE;
        }

        // Save tiles & return
        level.solidTiles = tiles;
        return level;
    }

    public boolean isEndReached(Player p) {
        // Flaggen-Hitbox skaliert
        Rectangle flagRect = new Rectangle(endX, (int) (endY - (150 * Zoom.SCALE)), (int) (40 * Zoom.SCALE), (int) (200 * Zoom.SCALE));
        return p.getBounds().intersects(flagRect);
    }

    public void draw(Graphics2D g, int camX) {
        // Hintergrund (Parallax-Effekt und Hintergrund-Wiederholung)
        if (background != null) {
            // Skalierte Breite und Höhe des Hintergrundbildes
            int bgWidth = (int) (background.getWidth() * Zoom.SCALE);
            int bgHeight = (int) (background.getHeight() * Zoom.SCALE);

            // 1. Berechne die Parallax-Verschiebung
            int parallaxShift = camX / 2;

            // 2. Berechne den Offset (damit die Kachelung bei 0 anfängt)
            // (Wir nutzen hier das Negativ, da die Kamera nach rechts wandert)
            int offset = -(parallaxShift % bgWidth);

            // 3. Schleife, die von links nach rechts zeichnet
            for (int x = offset; x < GamePanel.WIDTH; x += bgWidth) {

                // Zeichne das Bild an der Offset-Position (x),
                // die bereits die Kachel-Wiederholung und die Parallax-Verschiebung enthält.
                g.drawImage(background, x, 0, bgWidth, bgHeight, null);
            }
        } else {
            g.setColor(new Color(135, 206, 250));
            g.fillRect(0, 0, width, height);
        }


        // Boden
        for (Tile t : solidTiles) {
            t.draw(g, camX);
        }

        // Flagge: Skalieren aller Dimensionen
        int flagOffsetX = endX - camX;
        int poleBaseWidth = (int) (30 * Zoom.SCALE);
        int poleBaseHeight = (int) (10 * Zoom.SCALE);
        int poleMastWidth = (int) (5 * Zoom.SCALE);
        int poleMastHeight = (int) (60 * Zoom.SCALE);
        int flagWidth = (int) (20 * Zoom.SCALE);
        int flagHeight = (int) (15 * Zoom.SCALE);

        // Basis
        g.setColor(new Color(50, 50, 50));
        g.fillRect(flagOffsetX, endY + TILE_SIZE - poleBaseHeight, poleBaseWidth, poleBaseHeight);

        // Mast
        g.setColor(new Color(200, 200, 200));
        g.fillRect((int) (flagOffsetX + (12 * Zoom.SCALE)), endY + TILE_SIZE - poleBaseHeight - poleMastHeight, poleMastWidth, poleMastHeight);

        // Flagge
        g.setColor(Color.RED);
        g.fillRect((int) (flagOffsetX + (12 * Zoom.SCALE)), endY + TILE_SIZE - poleBaseHeight - poleMastHeight, flagWidth, flagHeight);
        g.setColor(Color.BLACK);
        g.drawRect((int) (flagOffsetX + (12 * Zoom.SCALE)), endY + TILE_SIZE - poleBaseHeight - poleMastHeight, flagWidth, flagHeight);
    }

    public ArrayList<Tile> getSolidTiles() {
        return solidTiles;
    }

    public int[][] getEnemyPositions() {
        return enemyPositions;
    }

    public int getWidth() {
        return width;
    }

    public int getHeight() {
        return height;
    }
}

In [ ]:
package levels;

import java.awt.*;
import java.awt.image.BufferedImage;

public class MovingPlatform extends Tile {

    private final double w, h;
    private final double startX, startY;
    private final double endX, endY;
    private final double speed;
    private double x, y;
    private int direction = 1; // nur eine Richtung entlang der Linie

    public MovingPlatform(double x, double y, double width, double height, double endX, double endY, double speed, BufferedImage texture) {
        super((int) x, (int) y, (int) width, (int) height, texture);
        this.x = x;
        this.y = y;
        this.w = width;
        this.h = height;
        this.startX = x;
        this.startY = y;
        this.endX = endX;
        this.endY = endY;
        this.speed = speed;
    }

    public void update(double dt) {
        // Vektor von Start zu End
        double dx = endX - startX;
        double dy = endY - startY;
        double dist = Math.sqrt(dx * dx + dy * dy);
        if (dist == 0) dist = 1;

        double normX = dx / dist;
        double normY = dy / dist;

        // Bewegung entlang der Linie
        double velX = normX * speed * direction * dt * 60;
        double velY = normY * speed * direction * dt * 60;

        x += velX;
        y += velY;

        // Prüfe Abstand von Startpunkt
        double proj = ((x - startX) * dx + (y - startY) * dy) / (dist * dist);
        if (proj >= 1) direction = -1;
        else if (proj <= 0) direction = 1;

        super.x = (int) Math.round(x);
        super.y = (int) Math.round(y);
    }

    @Override
    public void draw(Graphics2D g, int camX) {
        g.setColor(new Color(180, 180, 180, 150));
        int sx = (int) (startX - camX + w / 2);
        int sy = (int) (startY + h / 2);
        int ex = (int) (endX - camX + w / 2);
        int ey = (int) (endY + h / 2);
        g.setStroke(new BasicStroke(3));
        g.drawLine(sx, sy, ex, ey);

        g.drawImage(texture, (int) x - camX, (int) y, (int) w, (int) h, null);
    }
}

In [ ]:
package levels;

import java.awt.*;
import java.awt.image.BufferedImage;

public class Tile {
    public final BufferedImage texture;
    private final int w;
    private final int h;
    public int y;
    protected int x;

    public Tile(int x, int y, int w, int h, BufferedImage texture) {
        this.x = x;
        this.y = y;
        this.w = w;
        this.h = h;
        this.texture = texture;
    }

    public void draw(Graphics2D g, int camX) {
        if (texture != null) {
            g.drawImage(texture, x - camX, y, w, h, null);
        } else {
            g.setColor(new Color(0, 69, 168));
            g.fillRect(x - camX, y, w, h);
        }
    }

    public Rectangle getRect() {
        return new Rectangle(x, y, w, h);
    }

    public int getX() {
        return x;
    }

    public int getY() {
        return y;
    }

    public int getW() {
        return w;
    }

}

In [ ]:
package main;

import javax.swing.*;

public class Game {

    public static void createAndShowGUI() {
        JFrame frame = new JFrame("Super Jump Game");
        frame.setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);
        frame.setResizable(false);

        GamePanel panel = new GamePanel();
        frame.setContentPane(panel);
        frame.pack();
        frame.setLocationRelativeTo(null);
        frame.setVisible(true);

        panel.startGame();
    }
}

In [ ]:
package main;

import entities.Enemy;
import entities.MovingEnemy;
import entities.Player;
import levels.Level;
import levels.MovingPlatform;
import levels.Tile;
import ui.LoadingScreens;
import ui.MenuManager;
import utils.GameState;

import javax.swing.*;
import java.awt.*;
import java.awt.event.*;
import java.util.ArrayList;

public class GamePanel extends JPanel implements ActionListener {

    public static final int WIDTH = 800;
    public static final int HEIGHT = 600;
    public static GameState state = GameState.MENU_SCREEN;
    public static int lives = 3;
    public final LoadingScreens loadingScreens;
    private final MenuManager menuManager;
    private final Timer timer;
    private final Storage storage;
    private int currentLevelIndex;
    private int currentScore;
    private boolean left, right;
    private Player player;
    private ArrayList<Enemy> enemies;
    private ArrayList<MovingEnemy> movingEnemies;
    private Level level;
    private GameState nextStateAfterLoading;

    public GamePanel() {

        menuManager = new MenuManager(this);
        timer = new Timer(16, this);
        storage = new Storage();
        loadingScreens = new LoadingScreens();

        currentScore = 0;
        currentLevelIndex = 0;

        setPreferredSize(new Dimension(WIDTH, HEIGHT));
        setBackground(Color.CYAN);
        setFocusable(true);
        setupKeyBindings();
        setupMouse();
    }

    public static void subLive() {
        lives--;
    }

    public static void setlives(int lives) {
        GamePanel.lives = lives;
    }

    private void setupMouse() {
        addMouseListener(new MouseAdapter() {
            @Override
            public void mousePressed(MouseEvent e) {
                menuManager.handleMousePressed(e.getX(), e.getY());
            }
        });

        addMouseMotionListener(new MouseMotionAdapter() {
            @Override
            public void mouseMoved(MouseEvent e) {
                menuManager.setMousePosition(e.getX(), e.getY());
                repaint();
            }
        });
    }

    private void setupKeyBindings() {
        InputMap im = getInputMap(JComponent.WHEN_IN_FOCUSED_WINDOW);
        ActionMap am = getActionMap();

        /*
        im.put(KeyStroke.getKeyStroke("pressed ENTER"), "enter_pressed");
        am.put("enter_pressed", new AbstractAction() {
            @Override
            public void actionPerformed(ActionEvent e) {
                menuManager.handleEnter();
            }
        });
         */

        im.put(KeyStroke.getKeyStroke(KeyEvent.VK_ESCAPE, 0), "esc_pressed");
        am.put("esc_pressed", new AbstractAction() {
            @Override
            public void actionPerformed(ActionEvent e) {
                menuManager.handleEscape();
            }
        });

        // Links/Rechts Bewegung
        im.put(KeyStroke.getKeyStroke("pressed LEFT"), "left_pressed");
        im.put(KeyStroke.getKeyStroke("released LEFT"), "left_released");
        im.put(KeyStroke.getKeyStroke("pressed A"), "left_pressed");
        im.put(KeyStroke.getKeyStroke("released A"), "left_released");

        im.put(KeyStroke.getKeyStroke("pressed RIGHT"), "right_pressed");
        im.put(KeyStroke.getKeyStroke("released RIGHT"), "right_released");
        im.put(KeyStroke.getKeyStroke("pressed D"), "right_pressed");
        im.put(KeyStroke.getKeyStroke("released D"), "right_released");

        am.put("left_pressed", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                left = true;
            }
        });
        am.put("left_released", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                left = false;
            }
        });
        am.put("right_pressed", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                right = true;
            }
        });
        am.put("right_released", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                right = false;
            }
        });

        // Springen
        im.put(KeyStroke.getKeyStroke("pressed SPACE"), "jump_pressed");
        im.put(KeyStroke.getKeyStroke("released SPACE"), "jump_released");
        im.put(KeyStroke.getKeyStroke("pressed W"), "jump_pressed");
        im.put(KeyStroke.getKeyStroke("released W"), "jump_released");

        am.put("jump_pressed", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                if (player != null) player.pressJump();
            }
        });
        am.put("jump_released", new AbstractAction() {
            public void actionPerformed(ActionEvent e) {
                if (player != null) player.releaseJump();
            }
        });
    }

    public void startGame() {
        timer.start();
    }

    public void loadSelectedLevel(int index) {
        currentLevelIndex = index;
        level = Level.createSampleLevel(index);

        // Spieler mittig auf Boden spawnen
        int startX = Level.TILE_SIZE;
        int startY = 550 - Player.PLAYER_HEIGHT;

        player = new Player(startX, startY, level);
        player.loadSprite("res/player.png");

        enemies = new ArrayList<>();
        movingEnemies = new ArrayList<>();
        for (int[] p : level.getEnemyPositions()) {
            if (p[2] == 1) {
                movingEnemies.add(new MovingEnemy(p[0], p[1], level));
            } else {
                enemies.add(new Enemy(p[0], p[1], level));
            }
        }

        state = GameState.RUNNING;
        if (!timer.isRunning()) timer.start();
    }

    public int getCurrentLevelIndex() {
        return currentLevelIndex;
    }

    @Override
    public void actionPerformed(ActionEvent e) {
        double dt = timer.getDelay() / 1000.0;
        if (state == GameState.LOADING) {
            if (loadingScreens.isFinished()) {
                state = nextStateAfterLoading;
            }
            repaint();
            return;
        }


        if (state == GameState.RUNNING && player != null) {
            if (left && !right) player.moveLeft();
            else if (right && !left) player.moveRight();
            else player.stopHorizontal();

            for (Tile t : level.getSolidTiles()) {
                if (t instanceof MovingPlatform mp) mp.update(dt);
            }

            player.update(dt);

            for (Enemy en : new ArrayList<>(enemies)) en.update();
            for (Enemy en : new ArrayList<>(movingEnemies)) en.update();


            for (Enemy en : new ArrayList<>(enemies)) handleEnemyCollision(en);
            for (Enemy en : new ArrayList<>(movingEnemies)) handleEnemyCollision(en);


            if (level.isEndReached(player)) {
                state = GameState.LEVEL_COMPLETE_SCREEN;
                if (currentScore >= storage.getLevelHighscores(currentLevelIndex)) {
                    storage.setLevelHighscores(currentLevelIndex, currentScore);
                }
                storage.updateTotalScore();
            }
        }
        repaint();
    }

    @Override
    protected void paintComponent(Graphics g) {
        super.paintComponent(g);
        Graphics2D g2 = (Graphics2D) g;

        if (state == GameState.RUNNING || state == GameState.START_LEVEL_SCREEN || state == GameState.LEVEL_COMPLETE_SCREEN || state == GameState.GAME_OVER_SCREEN) {

            int camX = 0;
            if (player != null) {
                int px = player.getBounds().x;
                camX = px - WIDTH / 2;
                if (camX < 0) camX = 0;
                if (level != null && camX > level.getWidth() - WIDTH) camX = level.getWidth() - WIDTH;
            }

            if (level != null) level.draw(g2, camX);
            if (player != null) player.draw(g2, camX);
            if (movingEnemies != null) for (MovingEnemy en : movingEnemies) en.draw(g2, camX);
            if (enemies != null) for (Enemy en : enemies) en.draw(g2, camX);


            drawHUD(g2);
        }
        if (state == GameState.LOADING) {
            loadingScreens.draw(g2);
            return;
        }

        // Menü zeichnen
        menuManager.draw(g2);
    }

    private void drawHUD(Graphics2D g2) {
        g2.setColor(Color.BLACK);
        g2.setFont(new Font("SansSerif", Font.BOLD, 18));
        g2.drawString("Score: " + currentScore, 10, 20);
        g2.drawString("Lives: " + lives, 10, 40);
        g2.drawString("Level: " + (currentLevelIndex + 1), 700, 20);
    }

//    public void setGameState(GameState newState) {
//        state = newState;
//
//        if (state == GameState.MENU_SCREEN || state == GameState.LEVEL_SELECTION_SCREEN || state == GameState.CONTROLS_MENU_SCREEN) {
//            if (timer.isRunning()) timer.stop();
//        } else if (state == GameState.START_LEVEL_SCREEN || state == GameState.RUNNING) {
//            if (!timer.isRunning()) timer.start();
//        }
//    }

    public void showLoadingThen(GameState nextState) {
        nextStateAfterLoading = nextState;
        loadingScreens.start();
        state = GameState.LOADING;
    }


    public void restartLevel() {
        lives = 3;
        currentScore = 0;
        loadSelectedLevel(currentLevelIndex);
        state = GameState.START_LEVEL_SCREEN;
    }

    public int getCurrentScore() {
        return currentScore;
    }

    public void setCurrentScore(int currentScore) {
        this.currentScore = currentScore;
    }

    private void handleEnemyCollision(Enemy en) {
        if (!player.getBounds().intersects(en.getBounds())) return;

        // Spieler springt von oben auf den Gegner
        if (player.isFalling() && player.getY() < en.getY()) {
            removeEnemy(en);
            currentScore += 100;
            player.bounceAfterStomp();
            return;
        }

        // Spieler wird getroffen
        lives--;
        if (lives <= 0) state = GameState.GAME_OVER_SCREEN;
        else player.respawn();
    }

    private void removeEnemy(Enemy en) {
        enemies.remove(en);
        movingEnemies.remove(en);
    }

}

In [ ]:
package main;

import javax.swing.*;

public class Main {
    static void main() {
        SwingUtilities.invokeLater(Game::createAndShowGUI);
    }
}

In [ ]:
package main;

public class Storage {
    private static final int totalNumberOfLevels = 8;
    private static int totalScore;
    private int[] levelHighscores;

    public Storage() {
        totalScore = getTotalScore();
        levelHighscores = new int[totalNumberOfLevels];
    }

    public static int getTotalScore() {
        return totalScore;
    }

    public static void setTotalScore(int totalScore) {
        Storage.totalScore = totalScore;
    }

    public int getLevelHighscores(int currentLevelIndex) {
        return levelHighscores[currentLevelIndex];
    }

    public void setLevelHighscores(int levelIndex, int levelScore) {
        this.levelHighscores[levelIndex] = levelScore;
    }

    public int getTotalNumberOfLevels() {
        return totalNumberOfLevels;
    }

    public void updateTotalScore() {
        totalScore = 0;
        for (int i = totalNumberOfLevels -1; i >= 0; i--) {
            totalScore = totalScore + getLevelHighscores(i);
        }
    }
}

In [ ]:
package ui;

import main.GamePanel;

import java.awt.*;

public class LoadingScreens {
    private long startTime;
    private final long duration = 500; // 0.5 Sekunden Loading

    public void start() {
        startTime = System.currentTimeMillis();
    }

    public boolean isFinished() {
        return System.currentTimeMillis() - startTime >= duration;
    }

    public void draw(Graphics2D g) {

        g.setColor(Color.BLACK);
        g.fillRect(0, 0, GamePanel.WIDTH, GamePanel.HEIGHT);

        g.setColor(Color.WHITE);
        g.setFont(new Font("Arial", Font.BOLD, 40));

        String text = "Loading...";
        int w = g.getFontMetrics().stringWidth(text);
        g.drawString(text, (GamePanel.WIDTH - w) / 2, 250);

        // Optional: kleiner Balken
        int barWidth = 300;
        int barHeight = 20;
        int x = (GamePanel.WIDTH - barWidth) / 2;
        int y = 300;

        // Fortschritt berechnen
        double progress = (System.currentTimeMillis() - startTime) / (double) duration;
        if (progress > 1) progress = 1;

        g.setColor(Color.GRAY);
        g.fillRect(x, y, barWidth, barHeight);

        g.setColor(Color.GREEN);
        g.fillRect(x, y, (int) (barWidth * progress), barHeight);
    }
}

In [ ]:
package ui;

import main.GamePanel;
import main.Storage;
import utils.GameState;

import java.awt.*;
import java.util.ArrayList;
import java.util.List;

public class MenuManager {

    private final GamePanel game;
    private final List<Rectangle> levelButtons = new ArrayList<>();
    private final Rectangle startBtn;
    private final Rectangle controlsBtn;
    private final Rectangle levelSelectBtn;
    private final Rectangle quitBtn;
    private final Rectangle continueBtn;
    private final Rectangle restartBtn;
    private final Rectangle menuBtn;
    private int mouseX, mouseY;
    Storage storage;

    public MenuManager(GamePanel game) {
        this.game = game;
        storage = new Storage();

        // MainMenu Buttons
        startBtn = new Rectangle(300, 200, 200, 50);
        controlsBtn = new Rectangle(300, 270, 200, 50);
        levelSelectBtn = new Rectangle(300, 340, 200, 50);
        quitBtn = new Rectangle(300, 410, 200, 50);

        // LevelComplete Button
        continueBtn = new Rectangle(300, 250, 200, 50);

        // GameOver Button
        restartBtn = new Rectangle(300, 250, 200, 50);

        // Menu Button
        menuBtn = new Rectangle(300, 350, 200, 50);
    }

    public void setMousePosition(int x, int y) {
        mouseX = x;
        mouseY = y;
    }

    public void handleMousePressed(int mx, int my) {

        if (GamePanel.state == GameState.MENU_SCREEN) {
            GamePanel.setlives(3);
            if (startBtn.contains(mx, my)) {

                game.loadSelectedLevel(game.getCurrentLevelIndex());
                game.showLoadingThen(GameState.RUNNING);
            } else if (controlsBtn.contains(mx, my)) {
                game.showLoadingThen(GameState.CONTROLS_MENU_SCREEN);
            } else if (levelSelectBtn.contains(mx, my)) {
                game.showLoadingThen(GameState.LEVEL_SELECTION_SCREEN);
            } else if (quitBtn.contains(mx, my)) {
                System.exit(0);
            }
        } else if (GamePanel.state == GameState.LEVEL_SELECTION_SCREEN) {
            GamePanel.setlives(3);
            for (int i = 0; i < levelButtons.size(); i++) {
                if (levelButtons.get(i).contains(mx, my)) {
                    game.loadSelectedLevel(i);
                    game.showLoadingThen(GameState.RUNNING);
                    return;
                }
            }
        } else if (GamePanel.state == GameState.LEVEL_COMPLETE_SCREEN) {
            GamePanel.setlives(3);
            if (continueBtn.contains(mx, my)) {
                int next = game.getCurrentLevelIndex() + 1;
                if (next >= storage.getTotalNumberOfLevels()) {
                    game.showLoadingThen(GameState.GAME_OVER_SCREEN);
                    game.setCurrentScore(0);
                } else {
                    game.loadSelectedLevel(next);
                    game.showLoadingThen(GameState.RUNNING);
                }
            } else if (menuBtn.contains(mx, my)) {
                game.showLoadingThen(GameState.MENU_SCREEN);
                game.setCurrentScore(0);
            }
        } else if (GamePanel.state == GameState.GAME_OVER_SCREEN) {
            GamePanel.setlives(3);
            if (restartBtn.contains(mx, my)) {
                game.restartLevel();  // definiere diese Methode im main.GamePanel
            } else if (menuBtn.contains(mx, my)) {
                game.showLoadingThen(GameState.MENU_SCREEN);
            }
        }
    }

    /*
    public void handleEnter() {
        if (GamePanel.state == GameState.START_LEVEL_SCREEN) {
            game.showLoadingThen(GameState.RUNNING);
        }
    }
     */

    public void handleEscape() {
        if (GamePanel.state == GameState.RUNNING || GamePanel.state == GameState.START_LEVEL_SCREEN || GamePanel.state == GameState.LEVEL_COMPLETE_SCREEN || GamePanel.state == GameState.GAME_OVER_SCREEN || GamePanel.state == GameState.LEVEL_SELECTION_SCREEN || GamePanel.state == GameState.CONTROLS_MENU_SCREEN) {
            game.showLoadingThen(GameState.MENU_SCREEN);
        }
    }

    public void draw(Graphics2D g) {
        if (GamePanel.state == GameState.MENU_SCREEN) {
            drawMainMenu(g);
        } else if (GamePanel.state == GameState.LEVEL_SELECTION_SCREEN) {
            drawLevelSelection(g);
        } else if (GamePanel.state == GameState.CONTROLS_MENU_SCREEN) {
            drawControlsMenu(g);
        } else if (GamePanel.state == GameState.LEVEL_COMPLETE_SCREEN) {
            drawLevelComplete(g);
        } else if (GamePanel.state == GameState.GAME_OVER_SCREEN) {
            drawGameOver(g);
        } else if (GamePanel.state == GameState.START_LEVEL_SCREEN) {
            game.showLoadingThen(GameState.RUNNING);
        }

    }

    private void drawButton(Graphics2D g, Rectangle rect, String text) {
        if (rect.contains(mouseX, mouseY)) g.setColor(Color.ORANGE);
        else g.setColor(Color.LIGHT_GRAY);
        g.fill(rect);

        g.setColor(Color.BLACK);
        g.draw(rect);

        FontMetrics fm = g.getFontMetrics();
        int textX = rect.x + (rect.width - fm.stringWidth(text)) / 2;
        int textY = rect.y + (rect.height + fm.getAscent()) / 2 - 4;
        g.drawString(text, textX, textY);
    }

    private void drawMainMenu(Graphics2D g) {
        g.setColor(Color.DARK_GRAY);
        g.fillRect(0, 0, GamePanel.WIDTH, GamePanel.HEIGHT);

        g.setFont(new Font("Arial", Font.BOLD, 48));
        String title = "SUPER JUMP GAME";
        FontMetrics fm = g.getFontMetrics();
        int titleX = (GamePanel.WIDTH - fm.stringWidth(title)) / 2;
        g.setColor(Color.WHITE);
        g.drawString(title, titleX, 100);

        g.setFont(new Font("Arial", Font.PLAIN, 32));
        drawButton(g, startBtn, "Start");
        drawButton(g, controlsBtn, "Controls");
        drawButton(g, levelSelectBtn, "Level Select");
        drawButton(g, quitBtn, "Quit");

        g.setFont(new Font("SansSerif", Font.BOLD, 20));
        g.setColor(Color.YELLOW);
        g.drawString("Total Score: " + storage.getTotalScore(), 10, 50);
    }

    private void drawLevelSelection(Graphics2D g) {
        g.setColor(Color.DARK_GRAY);
        g.fillRect(0, 0, GamePanel.WIDTH, GamePanel.HEIGHT);

        g.setColor(Color.WHITE);
        g.setFont(new Font("Arial", Font.BOLD, 36));
        g.drawString("Level auswählen", 220, 100);

        levelButtons.clear();

        int columns = 4;
        int buttonWidth = 160;
        int buttonHeight = 60;
        int gap = 20;

        int totalLevels = storage.getTotalNumberOfLevels();
        int rows = (int) Math.ceil(totalLevels / (double) columns);

        int gridWidth = columns * (buttonWidth + gap) - gap;
        int startX = GamePanel.WIDTH / 2 - gridWidth / 2;
        int startY = 180;

        g.setFont(new Font("Arial", Font.PLAIN, 24));

        int index = 0;
        for (int row = 0; row < rows; row++) {
            for (int col = 0; col < columns; col++) {
                if (index >= totalLevels) break;
                int x = startX + col * (buttonWidth + gap);
                int y = startY + row * (buttonHeight + gap);

                Rectangle btn = new Rectangle(x, y, buttonWidth, buttonHeight);
                levelButtons.add(btn);

                drawButton(g, btn, "Level " + (index + 1));
                index++;
            }
        }
    }

    private void drawControlsMenu(Graphics2D g) {
        g.setColor(Color.DARK_GRAY);
        g.fillRect(0, 0, GamePanel.WIDTH, GamePanel.HEIGHT);

        g.setFont(new Font("Arial", Font.PLAIN, 28));
        g.setColor(Color.WHITE);

        String[] lines = {"Steuerung:", "Links/Rechts: Pfeiltasten oder A/D", "Springen: Leertaste oder W",};

        int totalHeight = lines.length * 40;
        int startY = (GamePanel.HEIGHT - totalHeight) / 2;

        for (int i = 0; i < lines.length; i++) {
            String line = lines[i];
            int x = (GamePanel.WIDTH - g.getFontMetrics().stringWidth(line)) / 2;
            int y = startY + i * 40;
            g.drawString(line, x, y);
        }

        String back = "Drücke ESC um zurückzugehen";
        int backX = (GamePanel.WIDTH - g.getFontMetrics().stringWidth(back)) / 2;
        g.drawString(back, backX, startY + lines.length * 40 + 20);
    }

    private void drawLevelComplete(Graphics2D g) {
        g.setColor(Color.BLACK);
        g.fillRect(0, 0, GamePanel.WIDTH, GamePanel.HEIGHT);

        g.setColor(Color.WHITE);
        g.setFont(new Font("Arial", Font.BOLD, 48));
        g.drawString("LEVEL COMPLETE", 200, 200);

        g.setColor(Color.YELLOW);
        g.setFont(new Font("Arial", Font.PLAIN, 24));
        g.drawString("YOUR SCORE WAS: " + game.getCurrentScore() + "!", 250, 225);

        g.setFont(new Font("Arial", Font.PLAIN, 32));
        drawButton(g, continueBtn, "Continue");
        drawButton(g, menuBtn, "Main Menu");
    }

    private void drawGameOver(Graphics2D g) {
        g.setColor(Color.BLACK);
        g.fillRect(0, 0, GamePanel.WIDTH, GamePanel.HEIGHT);

        // Titel
        g.setColor(Color.RED);
        g.setFont(new Font("Arial", Font.BOLD, 48));
        String title = "GAME OVER";
        FontMetrics fm = g.getFontMetrics();
        int titleX = (GamePanel.WIDTH - fm.stringWidth(title)) / 2;
        g.drawString(title, titleX, 100);

        // Total Score
        g.setFont(new Font("SansSerif", Font.BOLD, 24));
        g.setColor(Color.ORANGE);
        g.drawString("Total Score: " + Storage.getTotalScore(), 10, 50);

        // Buttons
        g.setFont(new Font("Arial", Font.PLAIN, 32));
        drawButton(g, restartBtn, "Restart ");
        drawButton(g, menuBtn, "Main Menu ");
    }
}

In [ ]:
package utils;

public enum GameState {
    MENU_SCREEN, START_LEVEL_SCREEN, RUNNING, LEVEL_COMPLETE_SCREEN, GAME_OVER_SCREEN, LEVEL_SELECTION_SCREEN, CONTROLS_MENU_SCREEN,
    LOADING, MENU_LOADING_SCREEN, START_LEVEL_LOADING_SCREEN, LEVEL_COMPLETE_LOADING_SCREEN, GAME_OVER_LOADING_SCREEN, LEVEL_SELECTION_LOADING_SCREEN, CONTROLS_MENU_LOADING_SCREEN
}

In [ ]:
package utils;

public class Zoom {
    public static final double SCALE = 1;
}